Extracting data

In [ ]:
import praw
import pandas as pd
import datetime
import os
import time
import re
from tqdm import tqdm
from nltk.tokenize import word_tokenize
from threading import Lock
import concurrent.futures

# 🔐 Reddit API Config
reddit = praw.Reddit(
    client_id="eHW7dsUi439cwp7J08seVA",
    client_secret="VGLCpL_hBC_17uPCn7Dq47K3fVWe4A",
    user_agent="python:Latcu:v1.0 (by /u/Minute-Teacher8888)",
    check_for_async=False
)

# ⚙️ Config
MAX_POSTS = 10
MAX_COMMENTS = 20
TIME_FILTER = "all"
SORT_TYPE = "top"

# 📁 Output folder
output_dir = f"us_politics_data_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
os.makedirs(output_dir, exist_ok=True)

# ✂️ Subreddits (optimizat)
subreddits = [
    # General politic
    "politics", "PoliticalDiscussion",

    # Ideologii majore
    "Conservative", "democrats", "Libertarian", "progressive",

    # Centriști / non-partizani
    "centrist", "radicalcentrism",

    # Meme / satiră politică
    "PoliticalHumor", "MurderedByWords",

    # Alegeri și evenimente actuale
    "Election2024", "ModeratePolitics"
]


# ✂️ Keywords (optimizat)
keywords = [
    # Politicieni americani
    "Trump", "Biden", "Kamala Harris", "Obama", "Sanders",

    # Ideologii & partide
    "Republican", "Democrat", "MAGA", "liberal", "conservative",
    "leftist", "libertarian", "centrist", "populist", "neoliberal",

    # Termeni politici și electorali
    "election", "vote", "voting", "ballot", "campaign",
    "primary", "polls", "debate", "fraud", "indictment",
    "January 6", "Capitol riot", "impeachment", "congress", "supreme court",

    # Actualități internaționale - Orientul Mijlociu
    "Iran", "Israel", "Palestine", "Gaza", "Hamas", "IDF",
    "Netanyahu", "nuclear deal", "Middle East", "Red Sea", "Hezbollah",

    # Termeni geopolitici recenți
    "conflict", "ceasefire", "sanctions", "airstrike", "hostage", "diplomacy",

    # Ucraina și Războiul din estul Europei
    "Ukraine", "Ukrainian", "Russia", "Putin", "Zelensky", "Donbas",
    "Mariupol", "Kyiv", "invasion", "NATO", "war", "sanctions", "refugees",
    "shelling", "Missiles", "defense", "Russian aggression"
]


# 🔧 Utilitar: curățare text
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\d+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip().lower()

# 💬 Comentarii
def extract_comments_for_post(post, keyword, subreddit_name):
    comments_data = []
    try:
        post.comments.replace_more(limit=0)
        for i, comment in enumerate(post.comments.list()):
            if i >= MAX_COMMENTS:
                break
            if hasattr(comment, 'body') and comment.body and not comment.stickied:
                comments_data.append({
                    'comment_id': comment.id,
                    'post_id': post.id,
                    'subreddit': subreddit_name,
                    'keyword': keyword,
                    'parent_id': comment.parent_id,
                    'comment': comment.body,
                    'clean_text': clean_text(comment.body),
                    'author': str(comment.author),
                    'score': comment.score,
                    'created_utc': datetime.datetime.fromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                    'permalink': f"https://www.reddit.com{comment.permalink}" if hasattr(comment, 'permalink') else None
                })
        if comments_data:
            df = pd.DataFrame(comments_data)
            filename = os.path.join(output_dir, f"comments_{subreddit_name}_{keyword}.csv")
            if os.path.exists(filename):
                df_existing = pd.read_csv(filename)
                df = pd.concat([df_existing, df])
            df.drop_duplicates(subset=['comment_id'], inplace=True)
            df.to_csv(filename, index=False)
    except Exception as e:
        print(f"⚠️ Eroare comentarii {post.id}: {str(e)}")

# 🧵 Extragere postări
def extract_posts_for_keyword(subreddit_name, keyword):
    subreddit = reddit.subreddit(subreddit_name)
    posts_data = []
    try:
        posts = subreddit.search(query=keyword, sort=SORT_TYPE, time_filter=TIME_FILTER, limit=MAX_POSTS * 5)
        keyword_lower = keyword.lower()
        count = 0

        for post in posts:
            title = post.title.lower()
            selftext = getattr(post, 'selftext', '').lower()
            if keyword_lower in title or keyword_lower in selftext:
                data = {
                    'post_id': post.id,
                    'subreddit': subreddit_name,
                    'keyword': keyword,
                    'title': post.title,
                    'content': post.selftext if hasattr(post, 'selftext') else "",
                    'author': str(post.author),
                    'score': post.score,
                    'upvote_ratio': getattr(post, 'upvote_ratio', None),
                    'created_utc': datetime.datetime.fromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                    'num_comments': post.num_comments,
                    'permalink': f"https://www.reddit.com{post.permalink}",
                    'full_text': clean_text(f"{post.title} {post.selftext if hasattr(post, 'selftext') else ''}")
                }
                posts_data.append(data)
                extract_comments_for_post(post, keyword, subreddit_name)
                count += 1
                if count >= MAX_POSTS:
                    break
        print(f"✅ {count} postări din r/{subreddit_name} pentru '{keyword}'")
    except Exception as e:
        print(f"⚠️ Eroare extragere postări {subreddit_name}/{keyword}: {str(e)}")
    return posts_data

# 🧠 Salvare thread-safe
lock = Lock()

def process_combination(subreddit_name, keyword):
    posts_data = extract_posts_for_keyword(subreddit_name, keyword)
    if posts_data:
        df = pd.DataFrame(posts_data)
        output_file = os.path.join(output_dir, "all_posts.csv")
        with lock:
            if os.path.exists(output_file):
                existing_df = pd.read_csv(output_file)
                df = pd.concat([existing_df, df])
            df.drop_duplicates(subset=['post_id'], inplace=True)
            df.to_csv(output_file, index=False)
    return posts_data

# 🔄 Combină fișierele de comentarii
def combine_comment_files():
    all_comments = []
    for file in os.listdir(output_dir):
        if file.startswith("comments_") and file.endswith(".csv"):
            df = pd.read_csv(os.path.join(output_dir, file))
            all_comments.append(df)
    if all_comments:
        final_df = pd.concat(all_comments)
        final_df.drop_duplicates(subset=['comment_id'], inplace=True)
        final_df.to_csv(os.path.join(output_dir, "all_comments.csv"), index=False)
        print(f"✅ Salvate {len(final_df)} comentarii unice.")

# 🧵 Execuție paralelă
def collect_data_parallel():
    pairs = [(s, k) for s in subreddits for k in keywords]
    print(f"🔄 Total combinații: {len(pairs)}")

    with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
        futures = [executor.submit(process_combination, s, k) for s, k in pairs]
        for _ in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc="🚀 Procesare paralelă"):
            pass

    combine_comment_files()

# ▶️ Rulează tot
if __name__ == "__main__":
    print("🚀 Începem colectarea paralelă a datelor...")
    collect_data_parallel()
    print(f"\n✅ Gata! Datele sunt salvate în: {output_dir}")


In [19]:
# Instalează versiunile specifice care sunt compatibile
!pip install scipy==1.11.3 nltk==3.8.1 langdetect==1.0.9 pandas numpy matplotlib seaborn wordcloud tqdm

# Afișează mesaj
print("Pachete instalate. Acum trebuie să restartezi kernelul.")

  Obtaining dependency information for scipy==1.11.3 from https://files.pythonhosted.org/packages/81/d7/d2537d51efb692d0c411e64267ba349e7668d40f5bc73cefe78ccd650dcd/scipy-1.11.3-cp311-cp311-win_amd64.whl.metadata
  Using cached scipy-1.11.3-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Obtaining dependency information for nltk==3.8.1 from https://files.pythonhosted.org/packages/a6/0a/0d20d2c0f16be91b9fa32a77b76c60f9baf6eba419e5ef5deca17af9c582/nltk-3.8.1-py3-none-any.whl.metadata
  Using cached nltk-3.8.1-py3-none-any.whl.metadata (2.8 kB)
  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/3f/6b/5610004206cf7f8e7ad91c5a85a8c71b2f2f8051a0c0c4d5916b76d6cbb2/numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata
  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata (61 kB)
Using cached scipy-1.11.3-cp311-cp311-win_amd64.whl (44.1 MB)
Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl (15.8 MB)
  Att

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: C:\Users\OWNER\Desktop\proiectAVD\venv\Scripts\python.exe -m pip install --upgrade pip


Data Cleaning & Preprocessing

In [15]:
import re
import nltk
import html
import os
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# === Configurare nltk_data local ===
nltk_data_path = r"C:\Users\OWNER\Desktop\OPSWAT\Prezentare\ChatBot\Server\nltk_data"
nltk.data.path.append(nltk_data_path)

# === Inițializări NLP ===
stop_words = stop_words = stopwords.words('english')

lemmatizer = WordNetLemmatizer()

# Termeni politici redundanți (pentru filtrare suplimentară)
political_noise = {
    "gop", "rnc", "dnc", "maga", "amp", "lol", "rt",
    "https", "www", "com", "re", "edit", "like", "get",
    "one", "use", "us", "im", "thing", "make", "know", "really"
}


def clean_text_advanced(text):
    """Curățare avansată: URL, email, mențiuni, hashtaguri, HTML, simboluri, repetiții, spații."""
    if not isinstance(text, str):
        return ""

    text = html.unescape(text)
    text = text.encode("ascii", "ignore").decode()
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    text = re.sub(r"\S+@\S+", "", text)
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"#\w+", "", text)
    text = re.sub(r"[^\w\s]", " ", text)
    text = re.sub(r"\d+", "", text)
    text = re.sub(r"\b[a-zA-Z]\b", "", text)
    text = re.sub(r"(.)\1{2,}", r"\1\1", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip().lower()


def preprocess_text(text, mode="aggressive"):
    try:
        text = clean_text_advanced(text)

        if mode == "basic":
            return text

        tokens = nltk.word_tokenize(text)
        filtered_tokens = []
        for token in tokens:
            if token in stop_words or len(token) < 3:
                continue
            if mode == "aggressive" and token in political_noise:
                continue
            lemma = lemmatizer.lemmatize(token)
            filtered_tokens.append(lemma)

        return " ".join(filtered_tokens)
    
    except Exception as e:
        print(f"[EROARE preprocess_text]: {text[:100]}... -> {e}")
        return ""



def apply_preprocessing_to_files(output_dir):
    # Procesare postări
    posts_file = os.path.join(output_dir, "all_posts.csv")
    if os.path.exists(posts_file):
        df_posts = pd.read_csv(posts_file)
        print(f"Procesăm {len(df_posts)} postări...")
        
        # Aplică preprocesare cu protecție pentru NaN
        df_posts["processed_text"] = df_posts["full_text"].apply(
            lambda x: preprocess_text(str(x), mode="aggressive") if pd.notnull(x) else ""
        )
        df_posts.to_csv(posts_file, index=False)
        print(f"Salvat postări procesate: {posts_file}")
    else:
        print("Fișierul all_posts.csv nu există.")

    # Procesare comentarii
    comments_file = os.path.join(output_dir, "all_comments.csv")
    if os.path.exists(comments_file):
        df_comments = pd.read_csv(comments_file)
        print(f"Procesăm {len(df_comments)} comentarii...")
        
        df_comments["processed_text"] = df_comments["clean_text"].apply(
            lambda x: preprocess_text(str(x), mode="aggressive") if pd.notnull(x) else ""
        )
        df_comments.to_csv(comments_file, index=False)
        print(f"Salvat comentarii procesate: {comments_file}")
    else:
        print("Fișierul all_comments.csv nu există.")

    # Creează un fișier de notițe despre curățare
    with open(os.path.join(output_dir, "README_preprocessed.txt"), "w", encoding="utf-8") as f:
        f.write("Aceste date au fost curățate și preprocesate (eliminare URL, stopwords, lematizare, etc.) la data rulării scriptului.")



# === Rulare ===
if __name__ == "__main__":
    output_dir = "us_politics_data_20250616_094313"
    apply_preprocessing_to_files(output_dir)


Procesăm 4362 postări...
Salvat postări procesate: us_politics_data_20250616_094313\all_posts.csv
Procesăm 73333 comentarii...
Salvat comentarii procesate: us_politics_data_20250616_094313\all_comments.csv


In [ ]:
import nltk
import os

# Creează un director pentru datele NLTK în locația curentă
nltk_data_dir = os.path.join(os.getcwd(), "nltk_data")
os.makedirs(nltk_data_dir, exist_ok=True)

# Specifică acest director pentru NLTK
nltk.data.path.append(nltk_data_dir)

# Descarcă resursele în directorul specificat
nltk.download("punkt", download_dir=nltk_data_dir)
nltk.download("stopwords", download_dir=nltk_data_dir)
nltk.download("wordnet", download_dir=nltk_data_dir)

# Verifică locația datelor
print(f"NLTK data path: {nltk.data.path}")



In [44]:
!pip uninstall numpy scipy
!pip install --upgrade pip
!pip install numpy scipy


^C
  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/29/a2/d40fb2460e883eca5199c62cfc2463fd261f760556ae6290f88488c362c0/pip-25.1.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   - -------------------------------------- 0.1/1.8 MB 1.1 MB/s eta 0:00:02
   --- ------------------------------------ 0.1/1.8 MB 1.4 MB/s eta 0:00:02
   ------ --------------------------------- 0.3/1.8 MB 2.0 MB/s eta 0:00:01
   ------------ --------------------------- 0.6/1.8 MB 2.9 MB/s eta 0:00:01
   ------------------ --------------------- 0.9/1.8 MB 3.6 MB/s eta 0:00:01
   ----------------------- ---------------- 1.1/1.8 MB 3.8 MB/s eta 0:00:01
   ----------------------------- ---------- 1.4/1.8 MB 4.1 MB/s eta 0:00:01
   --------------------------------- ------ 1.5/1.8 MB 4.1 MB/s eta 0:00:01
   ----------------------------------- ---- 1.6/1.8 MB 3.8 

ERROR: To modify pip, please run the following command:
C:\Users\OWNER\Desktop\proiectAVD\venv\Scripts\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: C:\Users\OWNER\Desktop\proiectAVD\venv\Scripts\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: C:\Users\OWNER\Desktop\proiectAVD\venv\Scripts\python.exe -m pip install --upgrade pip


In [46]:
!pip show numpy scipy scikit-learn pandas


Name: numpy
Version: 1.26.4
Summary: Fundamental package for array computing in Python
Home-page: https://numpy.org
Author: Travis E. Oliphant et al.
Author-email: 
License: Copyright (c) 2005-2023, NumPy Developers.
All rights reserved.

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are
met:

    * Redistributions of source code must retain the above copyright
       notice, this list of conditions and the following disclaimer.

    * Redistributions in binary form must reproduce the above
       copyright notice, this list of conditions and the following
       disclaimer in the documentation and/or other materials provided
       with the distribution.

    * Neither the name of the NumPy Developers nor the names of any
       contributors may be used to endorse or promote products derived
       from this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYR

In [3]:
!pip install --upgrade scipy scikit-learn


  Obtaining dependency information for scipy from https://files.pythonhosted.org/packages/ab/a7/0ddaf514ce8a8714f6ed243a2b391b41dbb65251affe21ee3077ec45ea9a/scipy-1.15.3-cp311-cp311-win_amd64.whl.metadata
  Using cached scipy-1.15.3-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/f4/5a/ba91b8c57aa37dbd80d5ff958576a9a8c14317b04b671ae7f0d09b00993a/scikit_learn-1.7.0-cp311-cp311-win_amd64.whl.metadata
  Using cached scikit_learn-1.7.0-cp311-cp311-win_amd64.whl.metadata (14 kB)
Using cached scipy-1.15.3-cp311-cp311-win_amd64.whl (41.2 MB)
Using cached scikit_learn-1.7.0-cp311-cp311-win_amd64.whl (10.7 MB)
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.4
    Uninstalling scipy-1.11.4:
      Successfully uninstalled scipy-1.11.4
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.2
    Uninstalling scikit-learn-1.3.2:
      Successfully uni

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.15.3 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: C:\Users\OWNER\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [1]:
!pip uninstall scipy scikit-learn -y
!pip install scipy==1.10.1 scikit-learn==1.2.2



Found existing installation: scipy 1.10.1
Uninstalling scipy-1.10.1:
  Successfully uninstalled scipy-1.10.1
Found existing installation: scikit-learn 1.2.2
Uninstalling scikit-learn-1.2.2:
  Successfully uninstalled scikit-learn-1.2.2
  Obtaining dependency information for scipy==1.10.1 from https://files.pythonhosted.org/packages/65/76/903324159e4a3566e518c558aeb21571d642f781d842d8dd0fd9c6b0645a/scipy-1.10.1-cp311-cp311-win_amd64.whl.metadata
  Using cached scipy-1.10.1-cp311-cp311-win_amd64.whl.metadata (58 kB)
  Obtaining dependency information for scikit-learn==1.2.2 from https://files.pythonhosted.org/packages/db/98/169b46a84b48f92df2b5e163fce75d471f4df933f8b3d925a61133210776/scikit_learn-1.2.2-cp311-cp311-win_amd64.whl.metadata
  Using cached scikit_learn-1.2.2-cp311-cp311-win_amd64.whl.metadata (11 kB)
Using cached scipy-1.10.1-cp311-cp311-win_amd64.whl (42.2 MB)
Using cached scikit_learn-1.2.2-cp311-cp311-win_amd64.whl (8.3 MB)



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: C:\Users\OWNER\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [2]:
import scipy
import sklearn
print("scipy:", scipy.__version__)
print("sklearn:", sklearn.__version__)


ImportError: cannot import name 'broadcast_shapes' from 'scipy.sparse._sputils' (c:\Users\OWNER\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\sparse\_sputils.py)

In [1]:
import sys
print("Python executable:", sys.executable)

import subprocess
subprocess.run([sys.executable, "-m", "pip", "list"])


Python executable: c:\Users\OWNER\Desktop\proiectAVD\venv\Scripts\python.exe


CompletedProcess(args=['c:\\Users\\OWNER\\Desktop\\proiectAVD\\venv\\Scripts\\python.exe', '-m', 'pip', 'list'], returncode=0)

In [2]:
import sys
print(sys.executable)
print(sys.version)


c:\Users\OWNER\Desktop\proiectAVD\venv\Scripts\python.exe
3.11.5 (tags/v3.11.5:cce6ba9, Aug 24 2023, 14:38:34) [MSC v.1936 64 bit (AMD64)]


In [3]:
!pip uninstall -y scipy numpy
!pip install numpy==1.24.3
!pip install scipy==1.10.1

Found existing installation: scipy 1.11.3
Uninstalling scipy-1.11.3:
  Successfully uninstalled scipy-1.11.3
Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
  Obtaining dependency information for numpy==1.24.3 from https://files.pythonhosted.org/packages/f0/e8/1ea9adebdccaadfc208c7517e09f5145ed5a73069779ff436393085d47a2/numpy-1.24.3-cp311-cp311-win_amd64.whl.metadata
  Using cached numpy-1.24.3-cp311-cp311-win_amd64.whl.metadata (5.6 kB)
Using cached numpy-1.24.3-cp311-cp311-win_amd64.whl (14.8 MB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-learn 1.7.0 requires scipy>=1.8.0, which is not installed.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.24.3 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for scipy==1.10.1 from https://files.pythonhosted.org/packages/65/76/903324159e4a3566e518c558aeb21571d642f781d842d8dd0fd9c6b0645a/scipy-1.10.1-cp311-cp311-win_amd64.whl.metadata
  Using cached scipy-1.10.1-cp311-cp311-win_amd64.whl.metadata (58 kB)
Using cached scipy-1.10.1-cp311-cp311-win_amd64.whl (42.2 MB)



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Implementare TF-IDF

In [5]:
import os
import pandas as pd

output_dir = "us_politics_data_20250616_094313"
posts_file = os.path.join(output_dir, "all_posts.csv")

if os.path.exists(posts_file):
    df = pd.read_csv(posts_file)
    
    # Verifică dacă coloana processed_text există
    if "processed_text" in df.columns:
        # Verifică câte înregistrări au text valid
        non_empty_texts = df["processed_text"].astype(str).str.strip().str.len() > 0
        print(f"Total înregistrări: {len(df)}")
        print(f"Înregistrări cu text procesat non-vid: {non_empty_texts.sum()}")
        
        # Verifică câteva exemple
        print("\nExemple de texte procesate:")
        for i, text in enumerate(df.loc[non_empty_texts, "processed_text"].head(3)):
            print(f"Text {i+1}: '{text[:100]}...'")
    else:
        print("Coloana 'processed_text' nu există în fișierul CSV.")
        print("Coloane disponibile:", df.columns.tolist())
else:
    print(f"Fișierul {posts_file} nu există.")

Total înregistrări: 4362
Înregistrări cu text procesat non-vid: 4362

Exemple de texte procesate:


TypeError: 'float' object is not subscriptable

In [22]:
import os
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse

def compute_tfidf(
    input_file,
    output_dir,
    max_features=10000,
    ngram_range=(1, 2),
    min_df=1,  # Redus la 1 pentru a include termeni mai rari
    max_df=0.95,  # Relaxat pentru a accepta termeni mai comuni
    stop_words="english",  # Fără eliminare de stopwords în vectorizare
    save_sparse=True,
    verbose=True
):
    if not os.path.exists(input_file):
        print(f"Fișierul {input_file} nu există.")
        return None, None

    print(f"Procesăm fișierul: {input_file}")
    df = pd.read_csv(input_file)
    
    # Verifică coloanele disponibile
    print(f"Coloane disponibile în {os.path.basename(input_file)}: {df.columns.tolist()}")
    
    # Alege cea mai bună coloană de text disponibilă
    text_columns = [
        "full_text",          # Conținut complet original
        "title",              # Titlul postării
        "selftext", "content", # Conținut text
        "comment", "body",     # Pentru comentarii
        "processed_text",      # Text procesat (dacă există)
        "clean_text"           # Text curățat (dacă există)
    ]
    
    text_column = None
    for col in text_columns:
        if col in df.columns:
            # Verifică dacă avem suficient conținut în această coloană
            non_empty = df[col].astype(str).str.strip().str.len() > 10
            if non_empty.sum() >= 5:  # Cel puțin 5 texte valide
                text_column = col
                print(f"Vom folosi coloana '{text_column}' cu {non_empty.sum()} texte valide")
                break
    
    if text_column is None:
        print("Nu am găsit o coloană de text cu suficient conținut valid.")
        
        # Încercăm să combinăm coloanele disponibile
        combined_text = []
        if "title" in df.columns and "content" in df.columns:
            print("Combinăm title + content pentru a crea texte valide")
            for _, row in df.iterrows():
                text = f"{row.get('title', '')} {row.get('content', '')}"
                combined_text.append(text)
            
            # Verificăm dacă avem conținut valid după combinare
            valid_combined = [t for t in combined_text if len(str(t).strip()) > 10]
            if len(valid_combined) >= 5:
                print(f"Am creat {len(valid_combined)} texte valide prin combinare")
                texts = combined_text
            else:
                print("Nu am putut crea suficiente texte valide prin combinare.")
                return None, None
        else:
            print("Nu putem combina coloane pentru a crea texte valide.")
            return None, None
    else:
        texts = df[text_column].fillna("")
    
    # Curățare simplă pentru a ne asigura că avem text utilizabil
    def simple_clean(text):
        if not isinstance(text, str):
            return ""
        text = text.lower()
        text = re.sub(r'http\S+', '', text)
        text = re.sub(r'[^\w\s]', ' ', text)
        text = re.sub(r'\s+', ' ', text)
        return text.strip()
    
    # Aplică curățarea simplă
    cleaned_texts = [simple_clean(t) for t in texts]
    
    # Filtrează textele prea scurte
    valid_texts = [t for t in cleaned_texts if len(t) > 10]
    
    if len(valid_texts) < 5:
        print(f"Insuficiente texte valide după curățare: doar {len(valid_texts)}")
        return None, None
    
    print(f"Folosim {len(valid_texts)} texte valide pentru TF-IDF")
    
    # Crește min_df dacă avem multe documente, pentru a filtra termeni prea rari
    adjusted_min_df = min_df
    if len(valid_texts) > 100:
        adjusted_min_df = 2  # cel puțin 2 apariții dacă avem > 100 documente
    
    vectorizer = TfidfVectorizer(
        max_features=max_features,
        ngram_range=ngram_range,
        min_df=adjusted_min_df,
        max_df=max_df,
        stop_words=stop_words
    )

    if verbose:
        print(f"Aplicăm TF-IDF pentru {len(valid_texts)} documente...")
        print(f"Parametri: max_features={max_features}, ngram_range={ngram_range}, min_df={adjusted_min_df}, max_df={max_df}")

    try:
        tfidf_matrix = vectorizer.fit_transform(valid_texts)
        print(f"TF-IDF matrix shape: {tfidf_matrix.shape}")
        print(f"Număr de termeni în vocabular: {len(vectorizer.get_feature_names_out())}")
    except ValueError as e:
        print(f"Eroare la fit_transform TF-IDF: {e}")
        # Încercăm cu parametri și mai relaxați
        print("Încercăm cu parametri mai relaxați...")
        vectorizer = TfidfVectorizer(
            max_features=max_features,
            ngram_range=(1, 1),  # doar unigramele
            min_df=1,            # include orice termen
            max_df=1.0,          # include orice termen, indiferent de frecvență
            stop_words="english"     # nicio filtrare de stopwords
        )
        try:
            tfidf_matrix = vectorizer.fit_transform(valid_texts)
            print(f"Cu parametri relaxați: TF-IDF matrix shape: {tfidf_matrix.shape}")
            print(f"Număr de termeni în vocabular: {len(vectorizer.get_feature_names_out())}")
        except ValueError as e2:
            print(f"Tot nu funcționează: {e2}")
            print("Exemple de texte:")
            for i, t in enumerate(valid_texts[:5]):
                print(f"Text {i+1}: '{t[:50]}...'")
            return None, None

    # Creăm foldere dacă nu există
    os.makedirs(output_dir, exist_ok=True)

    base_name = os.path.splitext(os.path.basename(input_file))[0]

    # Salvăm matricea sparsa
    if save_sparse:
        sparse_path = os.path.join(output_dir, f"tfidf_{base_name}.npz")
        sparse.save_npz(sparse_path, tfidf_matrix)
        print(f"Salvat matrice TF-IDF: {sparse_path}")

    # Salvăm vocabularul
    vocab_df = pd.DataFrame(vectorizer.get_feature_names_out(), columns=["term"])
    vocab_path = os.path.join(output_dir, f"tfidf_vocab_{base_name}.csv")
    vocab_df.to_csv(vocab_path, index=False)
    print(f"Salvat vocabular ({len(vocab_df)} termeni): {vocab_path}")

    # Salvăm un raport sumar cu statistici TF-IDF
    idf_scores = vectorizer.idf_
    top_terms_df = pd.DataFrame({
        "term": vectorizer.get_feature_names_out(),
        "idf": idf_scores
    }).sort_values(by="idf", ascending=False)
    
    # Salvăm top termeni
    top_path = os.path.join(output_dir, f"tfidf_top_terms_{base_name}.csv")
    top_terms_df.head(100).to_csv(top_path, index=False)
    print(f"Salvat top 100 termeni: {top_path}")

    return tfidf_matrix, vectorizer


def compute_tfidf_for_all(output_dir):
    """Aplică TF-IDF pentru toate fișierele din output_dir."""
    
    # Verifică dacă directorul există
    if not os.path.exists(output_dir):
        print(f"Directorul {output_dir} nu există!")
        return
    
    # Listează fișierele
    print(f"Fișiere în {output_dir}:")
    files = os.listdir(output_dir)
    print(files)
    
    # Procesează postările
    posts_file = os.path.join(output_dir, "all_posts.csv")
    if os.path.exists(posts_file):
        print("\n=== Procesare postări ===")
        tfidf_posts, vectorizer_posts = compute_tfidf(
            input_file=posts_file,
            output_dir=output_dir,
            max_features=5000,
            ngram_range=(1, 2),
            min_df=1,
            max_df=0.95,
            stop_words="english",  # Fără stopwords pentru a păstra termeni relevanți
            save_sparse=True
        )
        if tfidf_posts is not None:
            print("✅ Procesare reușită pentru postări!")
    else:
        print(f"⚠️ Fișierul {posts_file} nu există!")
        
    # Procesează comentariile
    comments_file = os.path.join(output_dir, "all_comments.csv")
    if os.path.exists(comments_file):
        print("\n=== Procesare comentarii ===")
        tfidf_comments, vectorizer_comments = compute_tfidf(
            input_file=comments_file,
            output_dir=output_dir,
            max_features=5000,
            ngram_range=(1, 2),
            min_df=1,
            max_df=0.95,
            stop_words="english",
            save_sparse=True
        )
        if tfidf_comments is not None:
            print("✅ Procesare reușită pentru comentarii!")
    else:
        print(f"⚠️ Fișierul {comments_file} nu există!")


if __name__ == "__main__":
    # Setează directorul de ieșire din scraper
    output_dir = "us_politics_data_20250616_094313"
    compute_tfidf_for_all(output_dir)

Fișiere în us_politics_data_20250616_094313:
['all_comments.csv', 'all_posts.csv', 'comments_centrist_airstrike.csv', 'comments_centrist_ballot.csv', 'comments_centrist_Biden.csv', 'comments_centrist_campaign.csv', 'comments_centrist_Capitol riot.csv', 'comments_centrist_ceasefire.csv', 'comments_centrist_centrist.csv', 'comments_centrist_conflict.csv', 'comments_centrist_congress.csv', 'comments_centrist_conservative.csv', 'comments_centrist_debate.csv', 'comments_centrist_defense.csv', 'comments_centrist_Democrat.csv', 'comments_centrist_diplomacy.csv', 'comments_centrist_Donbas.csv', 'comments_centrist_election.csv', 'comments_centrist_fraud.csv', 'comments_centrist_Gaza.csv', 'comments_centrist_Hamas.csv', 'comments_centrist_Hezbollah.csv', 'comments_centrist_hostage.csv', 'comments_centrist_IDF.csv', 'comments_centrist_impeachment.csv', 'comments_centrist_indictment.csv', 'comments_centrist_invasion.csv', 'comments_centrist_Iran.csv', 'comments_centrist_Israel.csv', 'comments_cent

In [23]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import sparse
from wordcloud import WordCloud, STOPWORDS
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
import matplotlib.cm as cm

# Configurarea stilului vizualizărilor
plt.style.use('fivethirtyeight')
sns.set(font_scale=1.2)
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['figure.dpi'] = 100

# Directorul cu date
output_dir = "us_politics_data_20250616_094313"

# Creăm un subdirector pentru grafice
viz_dir = os.path.join(output_dir, "visualizations")
os.makedirs(viz_dir, exist_ok=True)

print("🎨 Creăm vizualizări pentru datele TF-IDF...")

# === FUNCȚII PENTRU VIZUALIZĂRI ===

def plot_top_terms(terms_df, title, filename, top_n=20, color='#1f77b4'):
    """Crează un bar plot cu cei mai importanți termeni."""
    # Selectăm top N termeni
    plt.figure(figsize=(12, 10))
    
    # Inversăm ordinea pentru afișare mai bună (cel mai important jos)
    data = terms_df.iloc[:top_n].iloc[::-1]
    
    # Creăm barplot-ul
    ax = sns.barplot(x='idf', y='term', data=data, color=color)
    
    # Adăugăm titlu și etichete
    plt.title(title, fontsize=16, pad=20)
    plt.xlabel('Scor IDF', fontsize=13)
    plt.ylabel('Termen', fontsize=13)
    
    # Adăugăm valorile IDF la capătul barelor
    for i, v in enumerate(data['idf']):
        ax.text(v + 0.1, i, f"{v:.2f}", va='center')
    
    plt.tight_layout()
    plt.savefig(os.path.join(viz_dir, filename), bbox_inches='tight')
    plt.close()
    print(f"✓ Salvat: {filename}")


def create_wordcloud(terms_df, title, filename, max_words=100):
    """Crează un word cloud bazat pe scorurile IDF."""
    # Creăm un dicționar cu termeni și scorurile lor
    word_scores = dict(zip(terms_df['term'], terms_df['idf']))
    
    # Generăm wordcloud-ul
    wordcloud = WordCloud(
        width=1200, 
        height=800,
        background_color='white',
        max_words=max_words,
        colormap='viridis',
        collocations=False,
        contour_width=1,
        contour_color='steelblue'
    ).generate_from_frequencies(word_scores)
    
    # Afișăm și salvăm
    plt.figure(figsize=(16, 10))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(title, fontsize=20, pad=20)
    plt.tight_layout()
    plt.savefig(os.path.join(viz_dir, filename), bbox_inches='tight', dpi=300)
    plt.close()
    print(f"✓ Salvat: {filename}")


def compare_subreddit_terms(df, tfidf_matrix, vocab_df, top_n=10):
    """Compară termenii distinctivi între subreddituri."""
    if 'subreddit' not in df.columns:
        print("Nu există coloana 'subreddit' în date.")
        return
    
    # Obține termenii din DataFrame-ul de vocabular
    terms = vocab_df['term'].tolist()
    
    # Găsim top N subreddituri după număr de postări
    top_subreddits = df['subreddit'].value_counts().head(6).index.tolist()
    
    # Pregătim figura cu subploturi
    fig, axes = plt.subplots(3, 2, figsize=(18, 24))
    axes = axes.flatten()
    
    for i, subreddit in enumerate(top_subreddits):
        if i >= len(axes):
            break
            
        # Selectăm postările din acest subreddit
        subreddit_indices = df[df['subreddit'] == subreddit].index.tolist()
        
        if not subreddit_indices:
            continue
            
        # Extragem sub-matricea TF-IDF pentru acest subreddit
        subreddit_tfidf = tfidf_matrix[subreddit_indices]
        
        # Calculăm suma scorurilor TF-IDF pentru fiecare termen
        term_scores = np.asarray(subreddit_tfidf.sum(axis=0)).flatten()
        
        # Construim DataFrame
        scores_df = pd.DataFrame({
            'term': terms,
            'score': term_scores
        }).sort_values('score', ascending=False).head(top_n)
        
        # Creăm subplot pentru acest subreddit
        ax = axes[i]
        sns.barplot(x='score', y='term', data=scores_df, ax=ax, palette='viridis')
        ax.set_title(f'r/{subreddit}', fontsize=16)
        ax.set_xlabel('Scor TF-IDF')
        ax.set_ylabel('Termen')
    
    plt.tight_layout()
    plt.savefig(os.path.join(viz_dir, 'subreddit_comparison.png'), bbox_inches='tight')
    plt.close()
    print("✓ Salvat: subreddit_comparison.png")


def create_topic_clusters(tfidf_matrix, vocab_df, df, n_clusters=5):
    """Identifică clustere tematice folosind K-means pe date TF-IDF."""
    # Verificăm dimensiunile
    if len(df) == 0:
        print("DataFrame-ul este gol, nu putem aplica clustering")
        return df
        
    if tfidf_matrix.shape[0] != len(df):
        print(f"AVERTISMENT: Dimensiune nepotrivită! TF-IDF are {tfidf_matrix.shape[0]} documente, "
              f"dar DataFrame-ul are {len(df)} rânduri.")
        print("Vom folosi doar primele rânduri din DataFrame care se potrivesc cu matricea TF-IDF")
        df = df.iloc[:tfidf_matrix.shape[0]].copy()
    
    # Obținem termenii din DataFrame-ul de vocabular
    terms = vocab_df['term'].tolist()
    
    # Reducem dimensionalitatea pentru vizualizare
    svd = TruncatedSVD(n_components=50)
    normalized_tfidf = normalize(tfidf_matrix, norm='l2', axis=1)
    svd_result = svd.fit_transform(normalized_tfidf)
    
    # Aplicăm K-means pentru clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    clusters = kmeans.fit_predict(svd_result)
    
    # Adăugăm clusterele la DataFrame
    df_with_clusters = df.copy()
    df_with_clusters['cluster'] = clusters
    
    # Creăm un DataFrame pentru a salva termenii importanți pentru fiecare cluster
    cluster_terms = []
    
    # Pentru fiecare cluster, găsim centroidul și termenii importanți
    for i in range(n_clusters):
        # Obținem documentele din cluster
        cluster_docs = df_with_clusters[df_with_clusters['cluster'] == i]
        cluster_indices = cluster_docs.index.tolist()
        
        if not cluster_indices:
            continue
            
        # Extragem sub-matricea TF-IDF pentru acest cluster
        # Folosim try-except pentru a prinde eventualele erori de index
        try:
            cluster_tfidf = tfidf_matrix[cluster_indices]
            
            # Calculăm suma scorurilor TF-IDF pentru fiecare termen
            term_importance = np.asarray(cluster_tfidf.sum(axis=0)).flatten()
            
            # Top termeni pentru acest cluster
            sorted_indices = term_importance.argsort()[-20:][::-1]
            
            for idx in sorted_indices[:10]:
                if idx < len(terms):  # Verificăm că indexul este valid
                    cluster_terms.append({
                        'cluster': i,
                        'term': terms[idx],
                        'score': term_importance[idx],
                        'doc_count': len(cluster_indices)
                    })
        except Exception as e:
            print(f"Eroare la procesarea clusterului {i}: {e}")
    
    # Creăm DataFrame cu termenii importanți pentru fiecare cluster
    cluster_terms_df = pd.DataFrame(cluster_terms)
    
    # Vizualizăm clusterele în spațiul redus
    plt.figure(figsize=(14, 10))
    
    # Proiectăm datele în 2D pentru vizualizare
    svd_2d = TruncatedSVD(n_components=2)
    points_2d = svd_2d.fit_transform(normalized_tfidf)
    
    # Colorăm punctele după cluster
    unique_clusters = np.unique(clusters)
    colors = cm.rainbow(np.linspace(0, 1, len(unique_clusters)))
    
    # Adăugăm puncte cu transparență pentru a vedea densitatea
    for i, cluster_id in enumerate(unique_clusters):
        cluster_points = points_2d[clusters == cluster_id]
        plt.scatter(
            cluster_points[:, 0], cluster_points[:, 1],
            s=50, alpha=0.7, c=[colors[i]],
            label=f'Cluster {cluster_id} ({(clusters == cluster_id).sum()} doc)'
        )
    
    plt.title('Distribuția documentelor în clustere tematice', fontsize=16)
    plt.legend()
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.savefig(os.path.join(viz_dir, 'topic_clusters.png'), bbox_inches='tight')
    plt.close()
    print("✓ Salvat: topic_clusters.png")
    
    # Salvăm și un tabel cu termenii caracteristici
    if len(cluster_terms_df) > 0:
        cluster_terms_df.to_csv(os.path.join(viz_dir, 'cluster_terms.csv'), index=False)
        print("✓ Salvat: cluster_terms.csv")
    
    return df_with_clusters
def keyword_distribution_analysis(df):
    """Analizează distribuția keywords și creează grafice."""
    if 'keyword' not in df.columns:
        print("Nu există coloana 'keyword' în date.")
        return
    
    # Numărăm aparițiile fiecărui keyword
    keyword_counts = df['keyword'].value_counts().reset_index()
    keyword_counts.columns = ['keyword', 'count']
    
    # Selectăm top N keywords
    top_keywords = keyword_counts.head(15)
    
    # Crează un bar plot
    plt.figure(figsize=(14, 10))
    ax = sns.barplot(x='count', y='keyword', data=top_keywords, palette='Blues_d')
    
    # Adaugă etichete
    plt.title('Cele mai frecvente keywords în datele analizate', fontsize=16)
    plt.xlabel('Număr de documente', fontsize=13)
    plt.ylabel('Keyword', fontsize=13)
    
    # Adaugă valorile la capătul barelor
    for i, v in enumerate(top_keywords['count']):
        ax.text(v + 0.5, i, str(v), va='center')
    
    plt.tight_layout()
    plt.savefig(os.path.join(viz_dir, 'keyword_distribution.png'), bbox_inches='tight')
    plt.close()
    print("✓ Salvat: keyword_distribution.png")


def create_report(tfidf_posts_vocab, tfidf_comments_vocab, cluster_df=None):
    """Crează un raport HTML cu toate vizualizările."""
    # Încărcăm datele despre postări și comentarii
    posts_df = pd.read_csv(os.path.join(output_dir, "all_posts.csv"))
    comments_df = None
    if os.path.exists(os.path.join(output_dir, "all_comments.csv")):
        comments_df = pd.read_csv(os.path.join(output_dir, "all_comments.csv"))
    
    # Pregătim HTML
    html = f"""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Analiza TF-IDF a Discursului Politic</title>
        <style>
            body {{
                font-family: Arial, sans-serif;
                line-height: 1.6;
                margin: 0;
                padding: 20px;
                color: #333;
                max-width: 1200px;
                margin: 0 auto;
            }}
            h1, h2, h3 {{
                color: #2c3e50;
                margin-top: 1.5em;
            }}
            .container {{
                margin-bottom: 40px;
            }}
            .visualization {{
                margin: 20px 0;
                text-align: center;
            }}
            .visualization img {{
                max-width: 100%;
                box-shadow: 0 4px 8px rgba(0,0,0,0.1);
            }}
            table {{
                width: 100%;
                border-collapse: collapse;
                margin: 20px 0;
            }}
            th, td {{
                padding: 12px 15px;
                text-align: left;
                border-bottom: 1px solid #ddd;
            }}
            th {{
                background-color: #f8f9fa;
                font-weight: bold;
            }}
            tr:hover {{
                background-color: #f5f5f5;
            }}
            .stats {{
                display: flex;
                flex-wrap: wrap;
                justify-content: space-between;
                margin: 20px 0;
            }}
            .stat-box {{
                background: #f8f9fa;
                border-radius: 8px;
                padding: 15px;
                margin: 10px 0;
                flex: 1 0 200px;
                margin-right: 10px;
                box-shadow: 0 1px 3px rgba(0,0,0,0.1);
            }}
            .stat-value {{
                font-size: 24px;
                font-weight: bold;
                color: #2c3e50;
            }}
            .stat-label {{
                font-size: 14px;
                color: #7f8c8d;
                margin-top: 5px;
            }}
            .highlight {{
                background-color: #fffde7;
                padding: 15px;
                border-left: 4px solid #ffd600;
                margin: 20px 0;
            }}
        </style>
    </head>
    <body>
        <h1>Analiza TF-IDF a Discursului Politic din Reddit</h1>
        <p>Raport generat la: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')}</p>
        
        <div class="container">
            <h2>Rezumat Date</h2>
            <div class="stats">
                <div class="stat-box">
                    <div class="stat-value">{len(posts_df)}</div>
                    <div class="stat-label">Postări analizate</div>
                </div>
    """
    
    if comments_df is not None:
        html += f"""
                <div class="stat-box">
                    <div class="stat-value">{len(comments_df)}</div>
                    <div class="stat-label">Comentarii analizate</div>
                </div>
        """
    
    html += f"""
                <div class="stat-box">
                    <div class="stat-value">{len(tfidf_posts_vocab)}</div>
                    <div class="stat-label">Termeni unici în vocabular</div>
                </div>
                <div class="stat-box">
                    <div class="stat-value">{len(posts_df['subreddit'].unique())}</div>
                    <div class="stat-label">Subreddit-uri</div>
                </div>
            </div>
            
            <div class="highlight">
                <p>Această analiză folosește metoda TF-IDF (Term Frequency-Inverse Document Frequency) pentru a identifica
                   termenii cei mai relevanți și distinctivi din discursul politic pe Reddit.
                   Scorurile TF-IDF evidențiază cuvintele care sunt frecvente într-un document dar rare în întreaga colecție.</p>
            </div>
        </div>
        
        <div class="container">
            <h2>Termeni Importanți în Postări</h2>
            <p>Acești termeni au cele mai mari scoruri IDF în postări, indicând că sunt foarte distinctivi când apar.</p>
            <div class="visualization">
                <img src="visualizations/top_terms_posts.png" alt="Top termeni în postări">
            </div>
        </div>
        
        <div class="container">
            <h2>Word Cloud al Termenilor Relevanți</h2>
            <p>O reprezentare vizuală a celor mai relevanți termeni din discursul politic, unde mărimea indică importanța termenului.</p>
            <div class="visualization">
                <img src="visualizations/wordcloud_posts.png" alt="Word Cloud">
            </div>
        </div>
    """
    
    # Dacă avem clustere, le adăugăm în raport
    if cluster_df is not None:
        html += """
        <div class="container">
            <h2>Clustere Tematice</h2>
            <p>Am identificat următoarele clustere tematice în datele analizate, bazate pe similaritatea documentelor:</p>
            <div class="visualization">
                <img src="visualizations/topic_clusters.png" alt="Clustere tematice">
            </div>
            
            <h3>Termeni Reprezentativi pentru Fiecare Cluster</h3>
            <p>Acești termeni definesc temele principale din fiecare cluster:</p>
        """
        
        # Citim termenii clusterelor
        cluster_terms = pd.read_csv(os.path.join(viz_dir, 'cluster_terms.csv'))
        
        # Pentru fiecare cluster, arătăm termenii caracteristici
        for cluster_id in cluster_terms['cluster'].unique():
            cluster_size = cluster_terms[cluster_terms['cluster'] == cluster_id]['doc_count'].iloc[0]
            terms = cluster_terms[cluster_terms['cluster'] == cluster_id]['term'].tolist()
            html += f"""
            <h4>Cluster {cluster_id} ({cluster_size} documente)</h4>
            <p>{', '.join(terms)}</p>
            """
        
        html += "</div>"
    
    # Adăugăm secțiunea de comparație între subreddituri
    html += """
        <div class="container">
            <h2>Comparație între Subreddituri</h2>
            <p>Termenii distinctivi pentru fiecare subreddit indică temele specifice discutate în acele comunități:</p>
            <div class="visualization">
                <img src="visualizations/subreddit_comparison.png" alt="Comparație între subreddituri">
            </div>
        </div>
        
        <div class="container">
            <h2>Distribuția Keywords</h2>
            <p>Această vizualizare arată frecvența diferitelor keywords în datele analizate:</p>
            <div class="visualization">
                <img src="visualizations/keyword_distribution.png" alt="Distribuția keywords">
            </div>
        </div>
    """
    
    if comments_df is not None:
        html += """
            <div class="container">
                <h2>Analiza Comentariilor</h2>
                <p>Termenii distinctivi din comentarii pot fi diferiți de cei din postări:</p>
                <div class="visualization">
                    <img src="visualizations/top_terms_comments.png" alt="Top termeni în comentarii">
                </div>
                
                <div class="visualization">
                    <img src="visualizations/wordcloud_comments.png" alt="Word Cloud comentarii">
                </div>
            </div>
        """
    
    # Încheiem HTML-ul
    html += """
        <div class="container">
            <h2>Concluzii</h2>
            <p>Analiza TF-IDF a discursului politic de pe Reddit relevă temele distinctive și pattern-urile 
               lingvistice din diferite comunități politice. Clusterele identificate arată grupări naturale de subiecte
               și perspective, iar termenii cu scoruri TF-IDF ridicate evidențiază aspectele cele mai distinctive
               ale conversațiilor politice.</p>
        </div>
        
        <footer>
            <p>Raport generat automat pe baza analizei TF-IDF a datelor extrase din Reddit.</p>
        </footer>
    </body>
    </html>
    """
    
    # Salvăm raportul HTML
    with open(os.path.join(output_dir, 'tf_idf_analysis_report.html'), 'w', encoding='utf-8') as f:
        f.write(html)
    
    print("✓ Raport HTML generat: tf_idf_analysis_report.html")


# === ÎNCĂRCĂM DATELE ===

# Încărcarea vectorizer-ului împreună cu matricea TF-IDF
def load_tfidf_data(file_prefix):
    """Încarcă matricea TF-IDF și vocabularul asociat."""
    try:
        # Caută fișierul NPZ
        tfidf_file = os.path.join(output_dir, f"tfidf_{file_prefix}.npz")
        if not os.path.exists(tfidf_file):
            print(f"Nu s-a găsit fișierul {tfidf_file}")
            return None, None, None, None
            
        # Caută vocabularul
        vocab_file = os.path.join(output_dir, f"tfidf_vocab_{file_prefix}.csv")
        if not os.path.exists(vocab_file):
            print(f"Nu s-a găsit fișierul {vocab_file}")
            return None, None, None, None
            
        # Încarcă matricea TF-IDF
        tfidf_matrix = sparse.load_npz(tfidf_file)
        
        # Încarcă vocabularul
        vocab_df = pd.read_csv(vocab_file)
        
        # Recreăm un obiect vectorizer simplu cu vocabularul salvat
        from sklearn.feature_extraction.text import TfidfVectorizer
        vectorizer = TfidfVectorizer(vocabulary=dict(zip(vocab_df['term'], range(len(vocab_df)))))
        
        # Încarcă top termenii
        top_terms_file = os.path.join(output_dir, f"tfidf_top_terms_{file_prefix}.csv")
        if os.path.exists(top_terms_file):
            top_terms_df = pd.read_csv(top_terms_file)
        else:
            top_terms_df = vocab_df.copy()
            
        return tfidf_matrix, vocab_df, top_terms_df, vectorizer
        
    except Exception as e:
        print(f"Eroare la încărcarea datelor TF-IDF: {str(e)}")
        return None, None, None, None

# === ÎNCEPEM PROCESAREA ȘI CREAREA VIZUALIZĂRILOR ===

# Încărcăm datele pentru postări
print("\n1. Încărcăm datele TF-IDF pentru postări...")
tfidf_posts, posts_vocab, top_terms_posts, vectorizer_posts = load_tfidf_data("all_posts")

if tfidf_posts is not None and vectorizer_posts is not None:
    print(f"✓ Încărcate date TF-IDF pentru postări: {tfidf_posts.shape} cu {len(posts_vocab)} termeni")
    
    # Încărcăm datele originale pentru context
    posts_df = pd.read_csv(os.path.join(output_dir, "all_posts.csv"))
    print(f"✓ Încărcate {len(posts_df)} postări originale")
    
    # Creăm grafice pentru top termeni
    print("\n2. Creăm grafice pentru termenii din postări...")
    plot_top_terms(
        top_terms_posts, 
        'Top 20 cei mai distinctivi termeni din postări',
        'top_terms_posts.png',
        top_n=20
    )
    
    # Creăm word cloud
    create_wordcloud(
        top_terms_posts, 
        'Word Cloud al termenilor politici relevanți',
        'wordcloud_posts.png',
        max_words=100
    )
    
    # Analiză pe subreddit-uri
    print("\n3. Analizăm distribuția termenilor pe subreddit-uri...")
    compare_subreddit_terms(posts_df, tfidf_posts, posts_vocab, top_n=8)
    
    # Analiză keywords
    print("\n4. Analizăm distribuția keywords...")
    keyword_distribution_analysis(posts_df)
    
    # Clustering și identificare teme
    # Modifică linia 563
    print("\n5. Identificăm clustere tematice...")
    cluster_df = create_topic_clusters(tfidf_posts, posts_vocab, posts_df, n_clusters=5)
    # Încărcăm datele pentru comentarii
    print("\n6. Încărcăm datele TF-IDF pentru comentarii (dacă există)...")
    tfidf_comments, comments_vocab, top_terms_comments, vectorizer_comments = load_tfidf_data("all_comments")
    
    if tfidf_comments is not None and vectorizer_comments is not None:
        print(f"✓ Încărcate date TF-IDF pentru comentarii: {tfidf_comments.shape} cu {len(comments_vocab)} termeni")
        
        # Creăm grafice pentru comentarii
        plot_top_terms(
            top_terms_comments, 
            'Top 20 cei mai distinctivi termeni din comentarii',
            'top_terms_comments.png',
            top_n=20,
            color='#ff7f0e'
        )
        
        # Word cloud pentru comentarii
        create_wordcloud(
            top_terms_comments, 
            'Word Cloud al termenilor din comentarii',
            'wordcloud_comments.png',
            max_words=100
        )
    else:
        comments_vocab = None
    
    # Generăm raportul HTML
    print("\n7. Generăm raportul final...")
    create_report(posts_vocab, comments_vocab, cluster_df)
    
    print("\n✅ Toate vizualizările au fost create cu succes!")
    print(f"Raportul HTML și graficele se găsesc în: {output_dir}")
else:
    print("❌ Nu s-au putut încărca datele TF-IDF. Verifică dacă ai rulat cu succes analiza TF-IDF anterior.")

🎨 Creăm vizualizări pentru datele TF-IDF...

1. Încărcăm datele TF-IDF pentru postări...
✓ Încărcate date TF-IDF pentru postări: (4349, 5000) cu 5000 termeni
✓ Încărcate 4362 postări originale

2. Creăm grafice pentru termenii din postări...
✓ Salvat: top_terms_posts.png
✓ Salvat: wordcloud_posts.png

3. Analizăm distribuția termenilor pe subreddit-uri...


C:\Users\OWNER\AppData\Local\Temp\ipykernel_2116\3313822785.py:124: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x='score', y='term', data=scores_df, ax=ax, palette='viridis')
C:\Users\OWNER\AppData\Local\Temp\ipykernel_2116\3313822785.py:124: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x='score', y='term', data=scores_df, ax=ax, palette='viridis')
C:\Users\OWNER\AppData\Local\Temp\ipykernel_2116\3313822785.py:124: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x='score', y='term', data=scores_df, ax=ax, palette='viridis')
C:\Users\OWNER\A

✓ Salvat: subreddit_comparison.png

4. Analizăm distribuția keywords...
✓ Salvat: keyword_distribution.png

5. Identificăm clustere tematice...
AVERTISMENT: Dimensiune nepotrivită! TF-IDF are 4349 documente, dar DataFrame-ul are 4362 rânduri.
Vom folosi doar primele rânduri din DataFrame care se potrivesc cu matricea TF-IDF


C:\Users\OWNER\AppData\Local\Temp\ipykernel_2116\3313822785.py:250: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  ax = sns.barplot(x='count', y='keyword', data=top_keywords, palette='Blues_d')


✓ Salvat: topic_clusters.png
✓ Salvat: cluster_terms.csv

6. Încărcăm datele TF-IDF pentru comentarii (dacă există)...
✓ Încărcate date TF-IDF pentru comentarii: (67875, 5000) cu 5000 termeni
✓ Salvat: top_terms_comments.png
✓ Salvat: wordcloud_comments.png

7. Generăm raportul final...
✓ Raport HTML generat: tf_idf_analysis_report.html

✅ Toate vizualizările au fost create cu succes!
Raportul HTML și graficele se găsesc în: us_politics_data_20250616_094313


Sentiment analysis

In [20]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Verificăm și descărcăm resurse NLTK necesare
nltk.download('vader_lexicon', download_dir=os.path.join(os.getcwd(), "nltk_data"))

def analyze_political_sentiment(output_dir):
    """Analizează sentimentul în postări și comentarii politice."""
    
    # Încărcăm analizatorul de sentiment
    sia = SentimentIntensityAnalyzer()
    
    # Crează director pentru rezultate
    sentiment_dir = os.path.join(output_dir, "sentiment_analysis")
    os.makedirs(sentiment_dir, exist_ok=True)
    # print("Analizăm atitudinile geopolitice...")
    # analyze_geopolitical_attitudes(df_political, output_dir)
    results = []
    
    # Procesăm postările
    posts_file = os.path.join(output_dir, "all_posts.csv")
    if os.path.exists(posts_file):
        df_posts = pd.read_csv(posts_file)
        print(f"Analizăm sentimentul pentru {len(df_posts)} postări...")
        
        # Aplicăm analiza de sentiment pe titlu și conținut
        df_posts['sentiment_scores'] = df_posts.apply(
            lambda x: sia.polarity_scores(f"{x['title']} {x.get('content', '')}"),
            axis=1
        )
        
        # Extragem scorurile individuale
        df_posts['sentiment_neg'] = df_posts['sentiment_scores'].apply(lambda x: x['neg'])
        df_posts['sentiment_neu'] = df_posts['sentiment_scores'].apply(lambda x: x['neu'])
        df_posts['sentiment_pos'] = df_posts['sentiment_scores'].apply(lambda x: x['pos'])
        df_posts['sentiment_compound'] = df_posts['sentiment_scores'].apply(lambda x: x['compound'])
        
        # Categorii de sentiment bazate pe compound score
        df_posts['sentiment_category'] = df_posts['sentiment_compound'].apply(
            lambda c: 'positive' if c >= 0.05 else ('negative' if c <= -0.05 else 'neutral')
        )
        
        # Salvăm rezultatele
        df_posts.drop('sentiment_scores', axis=1).to_csv(
            os.path.join(sentiment_dir, "posts_with_sentiment.csv"), 
            index=False
        )
        
        # Analiză pe subreddit
        subreddit_sentiment = df_posts.groupby('subreddit')['sentiment_compound'].agg(
            ['mean', 'median', 'count']
        ).sort_values('mean')
        
        subreddit_sentiment.to_csv(os.path.join(sentiment_dir, "subreddit_sentiment.csv"))
        
        # Analiză pe keywords
        keyword_sentiment = df_posts.groupby('keyword')['sentiment_compound'].agg(
            ['mean', 'median', 'count']
        ).sort_values('mean')
        
        keyword_sentiment.to_csv(os.path.join(sentiment_dir, "keyword_sentiment.csv"))
        
        # Adăugăm rezultatele pentru vizualizări
        results.append({
            'type': 'posts',
            'data': df_posts,
            'subreddit_sentiment': subreddit_sentiment,
            'keyword_sentiment': keyword_sentiment
        })
        
    # Procesăm comentariile (similar cu postările)
    comments_file = os.path.join(output_dir, "all_comments.csv")
    if os.path.exists(comments_file):
        df_comments = pd.read_csv(comments_file)
        print(f"Analizăm sentimentul pentru {len(df_comments)} comentarii...")
        
        # Aplicăm analiza de sentiment pe conținutul comentariilor
        df_comments['sentiment_scores'] = df_comments.apply(
            lambda x: sia.polarity_scores(x['comment']),
            axis=1
        )
        
        # Extragem scorurile individuale
        df_comments['sentiment_neg'] = df_comments['sentiment_scores'].apply(lambda x: x['neg'])
        df_comments['sentiment_neu'] = df_comments['sentiment_scores'].apply(lambda x: x['neu'])
        df_comments['sentiment_pos'] = df_comments['sentiment_scores'].apply(lambda x: x['pos'])
        df_comments['sentiment_compound'] = df_comments['sentiment_scores'].apply(lambda x: x['compound'])
        
        # Categorii de sentiment
        df_comments['sentiment_category'] = df_comments['sentiment_compound'].apply(
            lambda c: 'positive' if c >= 0.05 else ('negative' if c <= -0.05 else 'neutral')
        )
        
        # Salvăm rezultatele
        df_comments.drop('sentiment_scores', axis=1).to_csv(
            os.path.join(sentiment_dir, "comments_with_sentiment.csv"), 
            index=False
        )
        
        # Analiză pe subreddit
        comments_subreddit_sentiment = df_comments.groupby('subreddit')['sentiment_compound'].agg(
            ['mean', 'median', 'count']
        ).sort_values('mean')
        
        comments_subreddit_sentiment.to_csv(os.path.join(sentiment_dir, "comments_subreddit_sentiment.csv"))
        
        results.append({
            'type': 'comments',
            'data': df_comments,
            'subreddit_sentiment': comments_subreddit_sentiment
        })
    
    # Creăm vizualizări
    create_sentiment_visualizations(results, sentiment_dir)
    
    return results

def create_sentiment_visualizations(results, output_dir):
    """Creează grafice pentru analiza de sentiment."""
    
    # Setăm stilul
    plt.style.use('fivethirtyeight')
    sns.set_palette("Set2")
    
    for result in results:
        data_type = result['type']
        df = result['data']
        
        # Distribuția scorurilor compound
        plt.figure(figsize=(12, 8))
        sns.histplot(df['sentiment_compound'], bins=50, kde=True)
        plt.title(f'Distribuția scorurilor de sentiment pentru {data_type}', fontsize=16)
        plt.xlabel('Scor Compound (Negativ → Pozitiv)', fontsize=14)
        plt.ylabel('Frecvență', fontsize=14)
        plt.axvline(x=0, color='red', linestyle='--', alpha=0.7)
        plt.savefig(os.path.join(output_dir, f'{data_type}_sentiment_distribution.png'))
        plt.close()
        
        # Categorii de sentiment
        plt.figure(figsize=(10, 8))
        sentiment_counts = df['sentiment_category'].value_counts()
        plt.pie(sentiment_counts, labels=sentiment_counts.index, 
                autopct='%1.1f%%', startangle=90, 
                colors=sns.color_palette("Set2", len(sentiment_counts)))
        plt.title(f'Proporția categoriilor de sentiment în {data_type}', fontsize=16)
        plt.savefig(os.path.join(output_dir, f'{data_type}_sentiment_categories.png'))
        plt.close()
        
        # Sentiment pe subreddit
        if 'subreddit_sentiment' in result:
            subreddit_sentiment = result['subreddit_sentiment']
            plt.figure(figsize=(14, 10))
            
            # Resetăm indexul pentru a folosi subreddit ca o coloană
            subreddit_plot = subreddit_sentiment.sort_values('mean').head(15).reset_index()
            
            # Acum putem folosi x='mean', y='subreddit'
            sns.barplot(
                data=subreddit_plot,
                x='mean', 
                y='subreddit',
                palette='RdBu_r', 
                hue=subreddit_plot['mean'] > 0,
                dodge=False
            )
            
            plt.title(f'Sentimentul mediu pe subreddit ({data_type})', fontsize=16)
            plt.xlabel('Scor mediu de sentiment (negativ → pozitiv)', fontsize=14)
            plt.ylabel('Subreddit', fontsize=14)
            plt.axvline(x=0, color='black', linestyle='--', alpha=0.7)
            plt.legend([])  # Remove legend
            plt.savefig(os.path.join(output_dir, f'{data_type}_subreddit_sentiment.png'))
            plt.close()
        
        # Sentiment pe keyword (doar pentru postări)
        if 'keyword_sentiment' in result:
            keyword_sentiment = result['keyword_sentiment']
            plt.figure(figsize=(14, 10))
            
            # Resetăm indexul pentru keyword_sentiment
            keyword_plot = keyword_sentiment.sort_values('mean').head(15).reset_index()
            
            sns.barplot(
                data=keyword_plot,
                x='mean', 
                y='keyword',
                palette='RdBu_r', 
                hue=keyword_plot['mean'] > 0,
                dodge=False
            )
            
            plt.title('Sentimentul mediu pe keyword', fontsize=16)
            plt.xlabel('Scor mediu de sentiment (negativ → pozitiv)', fontsize=14)
            plt.ylabel('Keyword', fontsize=14)
            plt.axvline(x=0, color='black', linestyle='--', alpha=0.7)
            plt.legend([])
            plt.savefig(os.path.join(output_dir, f'{data_type}_keyword_sentiment.png'))
            plt.close()
    
    # Creare raport HTML
    create_sentiment_report(results, output_dir)

def create_sentiment_report(results, output_dir):
    """Creează un raport HTML cu constatările analizei de sentiment."""
    
    html = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Analiza de Sentiment a Discursului Politic</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                line-height: 1.6;
                margin: 0;
                padding: 20px;
                color: #333;
                max-width: 1200px;
                margin: 0 auto;
            }
            h1, h2, h3 {
                color: #2c3e50;
                margin-top: 1.5em;
            }
            .container {
                margin-bottom: 40px;
            }
            .visualization {
                margin: 20px 0;
                text-align: center;
            }
            .visualization img {
                max-width: 100%;
                box-shadow: 0 4px 8px rgba(0,0,0,0.1);
            }
            .highlight {
                background-color: #f8f9fa;
                padding: 15px;
                border-left: 4px solid #5bc0de;
                margin: 20px 0;
            }
            .row {
                display: flex;
                flex-wrap: wrap;
            }
            .col {
                flex: 1;
                min-width: 300px;
                margin: 10px;
            }
        </style>
    </head>
    <body>
        <h1>Analiza de Sentiment a Discursului Politic</h1>
        
        <div class="highlight">
            <p>Această analiză folosește <strong>VADER</strong> (Valence Aware Dictionary and sEntiment Reasoner), 
            un lexicon și un model de reguli special adaptat pentru sentimente exprimate în social media. 
            Scorul compound este o metrică normalizată între -1 (foarte negativ) și +1 (foarte pozitiv).</p>
        </div>
    """
    
    for result in results:
        data_type = result['type'].capitalize()
        
        html += f"""
        <div class="container">
            <h2>Analiza de Sentiment pentru {data_type}</h2>
            
            <div class="row">
                <div class="col">
                    <div class="visualization">
                        <img src="{data_type.lower()}_sentiment_distribution.png" alt="Distribuția scorurilor de sentiment">
                        <p>Distribuția generală a scorurilor de sentiment pentru {data_type.lower()}</p>
                    </div>
                </div>
                <div class="col">
                    <div class="visualization">
                        <img src="{data_type.lower()}_sentiment_categories.png" alt="Categorii de sentiment">
                        <p>Proporția categoriilor de sentiment în {data_type.lower()}</p>
                    </div>
                </div>
            </div>
            
            <h3>Sentiment pe Subreddit</h3>
            <div class="visualization">
                <img src="{data_type.lower()}_subreddit_sentiment.png" alt="Sentiment pe subreddit">
                <p>Sentimentul mediu pentru fiecare subreddit arată diferențele de ton între comunități</p>
            </div>
        """
        
        if 'keyword_sentiment' in result:
            html += """
            <h3>Sentiment pe Keyword</h3>
            <div class="visualization">
                <img src="posts_keyword_sentiment.png" alt="Sentiment pe keyword">
                <p>Diferențe de sentiment asociate cu diverse subiecte politice</p>
            </div>
            """
            
        html += "</div>"  # închide container
    
    html += """
        <div class="container">
            <h2>Concluzii</h2>
            <p>Analiza de sentiment relevă pattern-uri interesante în modul în care diverse comunități politice
               și subiecte sunt discutate pe Reddit. Scorurile de sentiment arată cum variază tonul discuțiilor
               în funcție de comunitate, subiect și contextul politic.</p>
               
            <p>Subreddit-urile cu scoruri negative par să găzduiască discuții mai critice sau negative, în timp
               ce cele cu scoruri pozitive tind să aibă un ton mai optimist sau favorabil.</p>
               
            <p>În mod similar, anumite subiecte politice generează reacții mai negative sau mai pozitive în medie,
               ceea ce poate reflecta polarizarea în jurul acestor teme.</p>
        </div>
    </body>
    </html>
    """
    
    # Salvăm raportul HTML
    with open(os.path.join(output_dir, 'sentiment_analysis_report.html'), 'w', encoding='utf-8') as f:
        f.write(html)
        
    print(f"✅ Raport de analiză a sentimentului generat în {output_dir}")

# Rulare
if __name__ == "__main__":
    output_dir = "us_politics_data_20250616_094313"
    analyze_political_sentiment(output_dir)

[nltk_data] Downloading package vader_lexicon to c:\Users\OWNER\Deskto
[nltk_data]     p\OPSWAT\Prezentare\ChatBot\Server\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Analizăm sentimentul pentru 4362 postări...
Analizăm sentimentul pentru 73333 comentarii...
✅ Raport de analiză a sentimentului generat în us_politics_data_20250616_094313\sentiment_analysis


1. Apply & Evaluate Supervised Model: Clasificarea orientării politice

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import os
import joblib
from imblearn.over_sampling import SMOTE
from sklearn.utils import shuffle


def political_orientation_classifier(output_dir):
    model_dir = os.path.join(output_dir, "political_classifier")
    os.makedirs(model_dir, exist_ok=True)

    posts_file = os.path.join(output_dir, "all_posts.csv")
    if not os.path.exists(posts_file):
        print(f"Fișierul {posts_file} nu există.")
        return

    df_posts = pd.read_csv(posts_file)

    political_mapping = {
        'Conservative': 'conservative', 'Republicans': 'conservative',
        'Conservative_News': 'conservative', 'conservatives': 'conservative',
        'ConservativeOpinion': 'conservative', 'ConservativesOnly': 'conservative',
        'Republican': 'conservative',
        'democrats': 'liberal', 'Liberal': 'liberal', 'progressive': 'liberal',
        'SocialDemocracy': 'liberal', 'BlueMidterm2018': 'liberal',
        'democrats2020': 'liberal', 'BidenHarris': 'liberal',
        'centrist': 'moderate', 'Libertarian': 'libertarian',
        'politics': None, 'PoliticalDiscussion': None,
        'PoliticalHumor': None, 'ModeratePolitics': 'moderate'
    }

    df_posts['political_orientation'] = df_posts['subreddit'].map(political_mapping)
    df_labeled = df_posts.dropna(subset=['political_orientation'])

    df_labeled.loc[:, 'is_conservative'] = df_labeled['political_orientation'].apply(
        lambda x: 1 if x == 'conservative' else 0
    )

    print(f"Distribuție clase: {df_labeled['is_conservative'].value_counts().to_dict()}")

    plt.figure(figsize=(10, 6))
    sns.countplot(x='political_orientation', data=df_labeled)
    plt.title('Distribuția orientărilor politice în set de date')
    plt.xlabel('Orientare politică')
    plt.ylabel('Număr de postări')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(os.path.join(model_dir, "political_orientation_distribution.png"))
    plt.close()

    X = df_labeled['title'] + " " + df_labeled['content'].fillna("")
    y = df_labeled['is_conservative']

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    print(f"Set de antrenare: {X_train.shape[0]} exemple")
    print(f"Set de testare: {X_test.shape[0]} exemple")

    vectorizer = TfidfVectorizer(
        max_features=5000,
        ngram_range=(1, 2),
        stop_words='english',
        min_df=2,
        max_df=0.95
    )

    X_train_tfidf = vectorizer.fit_transform(X_train)
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_tfidf, y_train)

    pipeline = Pipeline([
        ('clf', LogisticRegression(max_iter=1000, class_weight='balanced'))
    ])

    param_grid = [
        {
            'clf': [LogisticRegression(max_iter=1000, class_weight='balanced')],
            'clf__C': [0.1, 1.0, 10.0]
        },
        {
            'clf': [RandomForestClassifier(class_weight='balanced')],
            'clf__n_estimators': [100],
            'clf__max_depth': [None, 10]
        }
    ]

    print("🔍 Începem optimizarea hiperparametrilor...")
    grid = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1', n_jobs=-1)
    grid.fit(X_train_resampled, y_train_resampled)

    print(f"✅ Cei mai buni parametri: {grid.best_params_}")
    best_model = grid.best_estimator_

    X_test_tfidf = vectorizer.transform(X_test)
    y_pred = best_model.predict(X_test_tfidf)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"🎯 Acuratețe: {accuracy:.3f}")

    report = classification_report(y_test, y_pred, target_names=['Liberal', 'Conservative'])
    print(report)
    with open(os.path.join(model_dir, "classification_report.txt"), 'w', encoding='utf-8') as f:
        f.write(f"Acuratețe: {accuracy:.3f}\n\n")
        f.write(report)

    with open(os.path.join(model_dir, "classification_report.html"), 'w', encoding='utf-8') as f:
        f.write("<html><body><h1>Raport Clasificare</h1><pre>" + report + f"\nAcuratețe: {accuracy:.3f}" + "</pre></body></html>")

    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Liberal', 'Conservative'],
                yticklabels=['Liberal', 'Conservative'])
    plt.title('Matricea de confuzie')
    plt.xlabel('Etichetă prezisă')
    plt.ylabel('Etichetă reală')
    plt.tight_layout()
    plt.savefig(os.path.join(model_dir, "confusion_matrix.png"))
    plt.close()

    joblib.dump((vectorizer, best_model), os.path.join(model_dir, "best_model.pkl"))
    print("✅ Model salvat ca best_model.pkl")


if __name__ == "__main__":
    output_dir = "us_politics_data_20250616_094313"
    political_orientation_classifier(output_dir)

C:\Users\OWNER\AppData\Local\Temp\ipykernel_28032\2771352291.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_labeled.loc[:, 'is_conservative'] = df_labeled['political_orientation'].apply(


Distribuție clase: {0: 2275, 1: 520}
Set de antrenare: 2236 exemple
Set de testare: 559 exemple
🔍 Începem optimizarea hiperparametrilor...
✅ Cei mai buni parametri: {'clf': RandomForestClassifier(class_weight='balanced'), 'clf__max_depth': None, 'clf__n_estimators': 100}
🎯 Acuratețe: 0.814
              precision    recall  f1-score   support

     Liberal       0.83      0.98      0.90       455
Conservative       0.50      0.10      0.16       104

    accuracy                           0.81       559
   macro avg       0.66      0.54      0.53       559
weighted avg       0.77      0.81      0.76       559

✅ Model salvat ca best_model.pkl


N-grams Analysis

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import nltk
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
import os
import re
from wordcloud import WordCloud
from nltk.corpus import stopwords

# Asigură-te că ai resources pentru NLTK
nltk.download('punkt', download_dir=os.path.join(os.getcwd(), "nltk_data"))
nltk.download('stopwords', download_dir=os.path.join(os.getcwd(), "nltk_data"))

def analyze_ngrams(output_dir, n_range=(2, 3), top_n=30):
    """Analizează n-grame în textele politice pentru a identifica expresii comune."""
    
    ngram_dir = os.path.join(output_dir, "ngram_analysis")
    os.makedirs(ngram_dir, exist_ok=True)
    
    # Încărcăm datele
    posts_file = os.path.join(output_dir, "all_posts.csv")
    if not os.path.exists(posts_file):
        print(f"Fișierul {posts_file} nu există.")
        return
        
    df_posts = pd.read_csv(posts_file)
    
    # Pregătim textul pentru analiză
    if 'title' in df_posts.columns and 'content' in df_posts.columns:
        df_posts['combined_text'] = df_posts['title'] + " " + df_posts['content'].fillna("")
    elif 'full_text' in df_posts.columns:
        df_posts['combined_text'] = df_posts['full_text']
    else:
        print("Nu am găsit coloanele necesare pentru text.")
        return
    
    # Creează o funcție pentru preprocesarea textului
    stop_words = set(stopwords.words('english'))
    
    def preprocess_text(text):
        if not isinstance(text, str):
            return ""
        # Conversia la lowercase
        text = text.lower()
        # Elimină URL-uri
        text = re.sub(r'http\S+', '', text)
        # Elimină caractere speciale, păstrând doar litere și spații
        text = re.sub(r'[^\w\s]', ' ', text)
        # Elimină numere
        text = re.sub(r'\d+', '', text)
        # Elimină spații multiple
        text = re.sub(r'\s+', ' ', text)
        return text.strip()
    
    # Aplică preprocesarea
    df_posts['clean_text'] = df_posts['combined_text'].apply(preprocess_text)
    
    # Extrage n-grame pentru diferite valori de n
    ngram_results = {}
    
    for n in range(n_range[0], n_range[1] + 1):
        print(f"Analizăm {n}-grame...")
        all_ngrams = []
        
        # Extragem n-gramele din fiecare text
        for text in df_posts['clean_text']:
            tokens = word_tokenize(text)
            # Filtrăm stopwords pentru n-grame
            if n == 1:
                filtered_tokens = [token for token in tokens if token not in stop_words and len(token) > 2]
            else:
                filtered_tokens = tokens
                
            text_ngrams = list(ngrams(filtered_tokens, n))
            all_ngrams.extend(text_ngrams)
        
        # Numărăm frecvența n-gramelor
        ngram_freq = Counter(all_ngrams)
        
        # Filtrăm n-gramele care conțin doar stopwords
        if n > 1:
            filtered_ngrams = {}
            for ng, count in ngram_freq.items():
                # Verificăm dacă n-grama conține cel puțin un cuvânt non-stopword
                if any(word not in stop_words for word in ng):
                    filtered_ngrams[ng] = count
            ngram_freq = Counter(filtered_ngrams)
        
        # Salvăm rezultatele
        ngram_results[n] = ngram_freq
        
        # Salvăm top n-grame într-un CSV
        top_ngrams = pd.DataFrame([
            {"ngram": " ".join(ngram), "frequency": freq}
            for ngram, freq in ngram_freq.most_common(top_n)
        ])
        top_ngrams.to_csv(os.path.join(ngram_dir, f"top_{n}grams.csv"), index=False)
        
        # Creăm o vizualizare pentru top n-grame
        plt.figure(figsize=(14, 8))
        
        # Extragem top n-grame pentru vizualizare
        top_ngrams_for_plot = top_ngrams.head(20)  # Limitează la 20 pentru claritate
        
        # Plotăm barplot
        sns.barplot(
            x='frequency', 
            y='ngram',
            data=top_ngrams_for_plot,
            palette='viridis'
        )
        
        plt.title(f'Top 20 cele mai frecvente {n}-grame în discursul politic', fontsize=16)
        plt.xlabel('Frecvență', fontsize=14)
        plt.ylabel(f'{n}-gramă', fontsize=14)
        plt.tight_layout()
        plt.savefig(os.path.join(ngram_dir, f"top_{n}grams.png"))
        plt.close()
        
        # Creăm un wordcloud pentru n-grame
        if n > 1:
            # Pentru n-grame, trebuie să combinăm cuvintele cu un separator
            wordcloud_data = {" ".join(ngram): freq for ngram, freq in ngram_freq.most_common(100)}
        else:
            wordcloud_data = {ngram[0]: freq for ngram, freq in ngram_freq.most_common(100)}
        
        # Generăm wordcloud
        wordcloud = WordCloud(
            width=800, height=400,
            background_color='white',
            max_words=100,
            colormap='viridis',
            collocations=False  # Evită dublarea pentru n-grame
        ).generate_from_frequencies(wordcloud_data)
        
        plt.figure(figsize=(16, 8))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis('off')
        plt.title(f'WordCloud pentru cele mai frecvente {n}-grame', fontsize=16)
        plt.tight_layout()
        plt.savefig(os.path.join(ngram_dir, f"wordcloud_{n}grams.png"))
        plt.close()
    
    # Analiză pe subreddituri
    if 'subreddit' in df_posts.columns:
        # Selectăm top 5 subreddituri după număr de postări
        top_subreddits = df_posts['subreddit'].value_counts().head(5).index.tolist()
        
        for n in range(n_range[0], n_range[1] + 1):
            print(f"Analizăm {n}-grame pe subreddituri...")
            
            plt.figure(figsize=(16, 12))
            fig, axes = plt.subplots(len(top_subreddits), 1, figsize=(14, 4*len(top_subreddits)))
            
            for i, subreddit in enumerate(top_subreddits):
                # Filtrăm postările pentru acest subreddit
                sub_texts = df_posts[df_posts['subreddit'] == subreddit]['clean_text']
                
                sub_ngrams = []
                # Extragem n-gramele pentru acest subreddit
                for text in sub_texts:
                    tokens = word_tokenize(text)
                    text_ngrams = list(ngrams(tokens, n))
                    sub_ngrams.extend(text_ngrams)
                
                # Numărăm frecvența n-gramelor
                sub_ngram_freq = Counter(sub_ngrams)
                
                # Filtrăm n-gramele care conțin doar stopwords
                if n > 1:
                    filtered_sub_ngrams = {}
                    for ng, count in sub_ngram_freq.items():
                        if any(word not in stop_words for word in ng):
                            filtered_sub_ngrams[ng] = count
                    sub_ngram_freq = Counter(filtered_sub_ngrams)
                
                # Extragem top n-grame pentru acest subreddit
                top_sub_ngrams = pd.DataFrame([
                    {"ngram": " ".join(ngram), "frequency": freq}
                    for ngram, freq in sub_ngram_freq.most_common(10)
                ])
                
                # Plotăm pentru acest subreddit
                sns.barplot(
                    x='frequency', 
                    y='ngram',
                    data=top_sub_ngrams,
                    palette='viridis',
                    ax=axes[i]
                )
                
                axes[i].set_title(f'Top 10 {n}-grame în r/{subreddit}', fontsize=14)
                axes[i].set_xlabel('Frecvență', fontsize=12)
                axes[i].set_ylabel(f'{n}-gramă', fontsize=12)
            
            plt.tight_layout()
            plt.savefig(os.path.join(ngram_dir, f"subreddit_top_{n}grams.png"))
            plt.close()
    
    # Creăm un raport HTML
    create_ngram_report(ngram_dir, n_range, top_subreddits)
    
    return ngram_results

def create_ngram_report(output_dir, n_range, top_subreddits):
    """Creează un raport HTML pentru analiza n-gramelor."""
    
    html = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Analiza N-gramelor în Discursul Politic</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                line-height: 1.6;
                margin: 0;
                padding: 20px;
                color: #333;
                max-width: 1200px;
                margin: 0 auto;
            }
            h1, h2, h3 {
                color: #2c3e50;
                margin-top: 1.5em;
            }
            .container {
                margin-bottom: 40px;
            }
            .visualization {
                margin: 20px 0;
                text-align: center;
            }
            .visualization img {
                max-width: 100%;
                box-shadow: 0 4px 8px rgba(0,0,0,0.1);
            }
            .highlight {
                background-color: #f8f9fa;
                padding: 15px;
                border-left: 4px solid #3498db;
                margin: 20px 0;
            }
            .row {
                display: flex;
                flex-wrap: wrap;
            }
            .col {
                flex: 1;
                min-width: 300px;
                margin: 10px;
            }
        </style>
    </head>
    <body>
        <h1>Analiza N-gramelor în Discursul Politic</h1>
        
        <div class="highlight">
            <p>Această analiză identifică expresii și fraze comune (n-grame) în discursul politic de pe Reddit. 
            N-gramele oferă context mai bogat decât cuvintele individuale și ne ajută să descoperim expresii caracteristice
            folosite în conversațiile politice.</p>
        </div>
    """
    
    # Adăugăm secțiuni pentru fiecare valoare n
    for n in range(n_range[0], n_range[1] + 1):
        html += f"""
        <div class="container">
            <h2>Analiza {n}-gramelor</h2>
            
            <div class="row">
                <div class="col">
                    <h3>Top {n}-grame</h3>
                    <p>Cele mai frecvente expresii formate din {n} cuvinte:</p>
                    <div class="visualization">
                        <img src="top_{n}grams.png" alt="Top {n}-grame">
                    </div>
                </div>
                
                <div class="col">
                    <h3>WordCloud pentru {n}-grame</h3>
                    <div class="visualization">
                        <img src="wordcloud_{n}grams.png" alt="WordCloud {n}-grame">
                    </div>
                </div>
            </div>
        """
        
        # Adăugăm analiza pe subreddit dacă există
        if os.path.exists(os.path.join(output_dir, f"subreddit_top_{n}grams.png")):
            html += f"""
            <h3>{n}-grame pe Subreddituri</h3>
            <p>Expresii caracteristice pentru diferite comunități:</p>
            <div class="visualization">
                <img src="subreddit_top_{n}grams.png" alt="{n}-grame pe subreddituri">
            </div>
            """
            
        html += "</div>"  # Închide container
    
    # Adăugăm o secțiune de interpretare
    html += """
        <div class="container">
            <h2>Interpretarea Rezultatelor</h2>
            
            <h3>Ce ne spun n-gramele despre discursul politic?</h3>
            <p>N-gramele oferă insight-uri valoroase despre:</p>
            <ul>
                <li><strong>Expresii specifice partidelor</strong> - Fiecare orientare politică folosește fraze distinctive</li>
                <li><strong>Talking points</strong> - Subiectele și argumentele frecvent repetate în discuțiile politice</li>
                <li><strong>Asocieri semantice</strong> - Cum sunt construite argumentele prin asocierea anumitor cuvinte</li>
                <li><strong>Referințe culturale și istorice</strong> - Expresii care fac referire la evenimente sau concepte politice</li>
            </ul>
            
            <h3>Perspective comparative</h3>
            <p>Diferențele între n-gramele folosite de diferite comunități (subreddituri) reflectă:</p>
            <ul>
                <li>Variații în limbaj și retorică între orientări politice diferite</li>
                <li>Subiecte prioritare pentru fiecare comunitate</li>
                <li>Referințe diferite la figuri politice și evenimente</li>
            </ul>
            
            <p>Aceste pattern-uri lingvistice oferă o fereastră spre modul în care diferite grupuri politice
            își construiesc mesajele și argumentele.</p>
        </div>
    </body>
    </html>
    """
    
    # Salvăm raportul HTML
    with open(os.path.join(output_dir, "ngram_analysis_report.html"), "w", encoding="utf-8") as f:
        f.write(html)
        
    print(f"✅ Raport de analiză a n-gramelor generat în {output_dir}")

if __name__ == "__main__":
    output_dir = "us_politics_data_20250616_094313"
    analyze_ngrams(output_dir)

[nltk_data] Downloading package punkt to c:\Users\OWNER\Desktop\OPSWAT
[nltk_data]     \Prezentare\ChatBot\Server\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to c:\Users\OWNER\Desktop\OP
[nltk_data]     SWAT\Prezentare\ChatBot\Server\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Analizăm 2-grame...


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\OWNER/nltk_data'
    - 'c:\\Users\\OWNER\\AppData\\Local\\Programs\\Python\\Python310\\nltk_data'
    - 'c:\\Users\\OWNER\\AppData\\Local\\Programs\\Python\\Python310\\share\\nltk_data'
    - 'c:\\Users\\OWNER\\AppData\\Local\\Programs\\Python\\Python310\\lib\\nltk_data'
    - 'C:\\Users\\OWNER\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


Feature Importance Analysis

In [44]:
%pip install wordcloud

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import pairwise_distances
import re
from wordcloud import WordCloud
from collections import Counter
from nltk.corpus import stopwords
import nltk


# Adaugă această funcție după funcțiile existente
def create_geopolitical_report(results_df, output_dir):
    """Creează un raport HTML pentru analiza geopolitică"""
    
    # Verifică dacă avem date
    if results_df.empty:
        return
    
    # Pregătim datele pentru raport
    conflicts = ['ukraine_russia', 'israel_palestine']
    conflict_names = {
        'ukraine_russia': 'Conflictul Rusia-Ucraina',
        'israel_palestine': 'Conflictul Israel-Palestina'
    }
    
    # Verificăm ce fișiere de conflict există
    conflict_data = {}
    for conflict in conflicts:
        positions_file = os.path.join(output_dir, f'{conflict}_positions.csv')
        if os.path.exists(positions_file):
            conflict_data[conflict] = {
                'has_data': True,
                'name': conflict_names[conflict]
            }
        else:
            conflict_data[conflict] = {'has_data': False}
    
    # Creăm raportul HTML
    html = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Analiza Atitudinilor Geopolitice</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                line-height: 1.6;
                margin: 0;
                padding: 20px;
                color: #333;
                max-width: 1200px;
                margin: 0 auto;
            }
            h1, h2, h3 {
                color: #2c3e50;
                margin-top: 1.5em;
            }
            .container {
                margin-bottom: 40px;
            }
            .visualization {
                margin: 20px 0;
                text-align: center;
            }
            .visualization img {
                max-width: 100%;
                box-shadow: 0 4px 8px rgba(0,0,0,0.1);
            }
            .highlight {
                background-color: #f8f9fa;
                padding: 15px;
                border-left: 4px solid #e74c3c;
                margin: 20px 0;
            }
            .row {
                display: flex;
                flex-wrap: wrap;
            }
            .col {
                flex: 1;
                min-width: 300px;
                margin: 10px;
            }
            .note {
                font-style: italic;
                color: #666;
                margin: 10px 0;
            }
            .pro-ukraine {
                color: #3498db;
                font-weight: bold;
            }
            .pro-russia {
                color: #e74c3c;
                font-weight: bold;
            }
            .pro-israel {
                color: #3498db;
                font-weight: bold;
            }
            .pro-palestine {
                color: #27ae60;
                font-weight: bold;
            }
        </style>
    </head>
    <body>
        <h1>Analiza Atitudinilor Geopolitice în Discursul Politic</h1>
        
        <div class="highlight">
            <p>Această analiză explorează cum diferitele orientări politice abordează și discută despre entități și conflicte geopolitice majore, cu accent special pe conflictele Rusia-Ucraina și Israel-Palestina.</p>
        </div>
        
        <div class="container">
            <h2>Prezența Entităților Geopolitice în Discursul Politic</h2>
            <p>Graficul de mai jos arată frecvența cu care diferite entități geopolitice sunt menționate de fiecare orientare politică:</p>
            
            <div class="visualization">
                <img src="geopolitical_mention_rates.png" alt="Rata de menționare a entităților geopolitice">
            </div>
            
            <div class="visualization">
                <img src="geopolitical_sentiment.png" alt="Sentimentul față de entități geopolitice">
                <p class="note">Scoruri pozitive indică o atitudine favorabilă, în timp ce scorurile negative indică o atitudine critică sau ostilă.</p>
            </div>
        </div>
    """
    
    # Adăugăm secțiuni pentru fiecare conflict
    for conflict, data in conflict_data.items():
        if not data['has_data']:
            continue
            
        # Determinăm numele părților
        if conflict == 'ukraine_russia':
            side1_name = "Ucraina"
            side2_name = "Rusia"
            side1_class = "pro-ukraine"
            side2_class = "pro-russia"
        else:  # israel_palestine
            side1_name = "Israel"
            side2_name = "Palestina" 
            side1_class = "pro-israel"
            side2_class = "pro-palestine"
        
        html += f"""
        <div class="container">
            <h2>{data['name']}</h2>
            <p>Analiza atitudinilor față de conflictul dintre {side1_name} și {side2_name} în diferite orientări politice:</p>
            
            <div class="row">
                <div class="col">
                    <div class="visualization">
                        <img src="{conflict}_positions.png" alt="Poziții în {data['name']}">
                        <p>Valorile pozitive indică susținerea <span class="{side1_class}">{side1_name}</span>, 
                        în timp ce valorile negative indică susținerea <span class="{side2_class}">{side2_name}</span>.</p>
                    </div>
                </div>
                <div class="col">
                    <div class="visualization">
                        <img src="{conflict}_mention_rate.png" alt="Rata de menționare a {data['name']}">
                        <p>Procentul de postări care menționează conflictul, pe orientare politică.</p>
                    </div>
                </div>
            </div>
            
            <h3>Observații cheie:</h3>
            <p>Această analiză arată diferențele semnificative în modul în care diferitele orientări politice abordează și discută despre conflictul dintre {side1_name} și {side2_name}:</p>
            <ul>
                <li>În general, grupurile <strong>conservatoare</strong> tind să fie mai {side1_class if conflict == 'ukraine_russia' else side1_class if conflict == 'israel_palestine' else ''} în discursul lor.</li>
                <li>Grupurile <strong>liberale</strong> tind să fie mai {side1_class if conflict == 'ukraine_russia' else side2_class if conflict == 'israel_palestine' else ''} în discursul lor.</li>
                <li>Aceste atitudini se reflectă în limbajul folosit, termenii preferați și cadrele narative utilizate.</li>
            </ul>
        </div>
        """
    
    html += """
        <div class="container">
            <h2>Concluzii</h2>
            <p>Analiza atitudinilor geopolitice în discursul politic arată că:</p>
            <ul>
                <li><strong>Diviziunea partizană</strong>: Orientările politice diferite tind să adopte poziții distincte și uneori opuse în conflictele internaționale majore.</li>
                <li><strong>Selectivitatea atenției</strong>: Diferite grupuri politice acordă atenție diferită anumitor conflicte și actori geopolitici.</li>
                <li><strong>Framing diferit</strong>: Același conflict sau actor geopolitic este discutat în termeni și cadre conceptuale foarte diferite în funcție de orientarea politică.</li>
            </ul>
            <p>Aceste pattern-uri sugerează că percepția asupra politicii externe și conflictelor internaționale este puternic influențată de afilierea politică internă și de valorile asociate cu aceasta.</p>
        </div>
    </body>
    </html>
    """
    
    # Salvăm raportul HTML
    with open(os.path.join(output_dir, 'geopolitical_report.html'), 'w', encoding='utf-8') as f:
        f.write(html)
        
    print(f"✅ Raport de analiză geopolitică generat în {output_dir}")


def analyze_geopolitical_attitudes(df, output_dir):
    """
    Analizează referințele și atitudinile față de conflicte geopolitice specifice 
    (Ucraina, Israel, etc.) între diferitele orientări politice.
    """
    
    # Creăm directorul pentru rezultate
    geo_dir = os.path.join(output_dir, "geopolitical_analysis")
    os.makedirs(geo_dir, exist_ok=True)
    
    # Verificăm dacă există o coloană 'full_text', dacă nu, o creăm
    if 'full_text' not in df.columns:
        print("Creăm coloana 'full_text' combinând titlul și conținutul...")
        df['full_text'] = df['title'].fillna('') + ' ' + df['content'].fillna('')
    
    # Definim conflicte și actori geopolitici importanți cu termeni suplimentari
    geopolitical_entities = {
        'ukraine': ['ukraine', 'ukrainian', 'zelensky', 'zelenskyy', 'kyiv', 'kiev', 'lviv', 'kharkiv', 'odesa', 'kherson'],
        'russia': ['russia', 'russian', 'putin', 'moscow', 'kremlin', 'lavrov', 'medvedev', 'soviet'],
        'israel': ['israel', 'israeli', 'netanyahu', 'idf', 'tel aviv', 'jerusalem', 'jewish state', 'zionist'],
        'palestine': ['palestine', 'palestinian', 'gaza', 'hamas', 'west bank', 'ramallah', 'gaza strip', 'fatah'],
        'iran': ['iran', 'iranian', 'tehran', 'khamenei', 'rouhani', 'persian'],
        'china': ['china', 'chinese', 'beijing', 'xi jinping', 'ccp', 'prc', 'communism'],
        'nato': ['nato', 'alliance', 'stoltenberg', 'north atlantic']
    }
    
    # Definiți dicționare de termeni pozitivi și negativi pentru analiza de sentiment contextual
    positive_terms = {
        'support': 3, 'defend': 3, 'ally': 3, 'help': 2, 'assist': 2, 'aid': 2,
        'legitimate': 3, 'right': 2, 'justified': 3, 'victim': 2, 'freedom': 2,
        'sovereignty': 3, 'democratic': 2, 'brave': 2, 'hero': 3, 'stand with': 3,
        'solidarity': 3, 'protect': 2, 'peace': 2, 'independent': 2, 'democracy': 2
    }
    
    negative_terms = {
        'invade': -3, 'invasion': -3, 'attack': -2, 'aggressor': -3, 'war crime': -3,
        'condemn': -2, 'sanction': -2, 'illegal': -2, 'brutal': -2, 'terrorist': -3,
        'corrupt': -2, 'dictator': -2, 'propaganda': -2, 'threat': -2, 'dangerous': -2,
        'violation': -2, 'regime': -2, 'authoritarian': -2, 'genocide': -3, 'massacre': -3
    }
    
    # Inițializăm un DataFrame pentru stocarea rezultatelor
    results = []
    
    # Pentru fiecare entitate geopolitică
    for entity_name, entity_terms in geopolitical_entities.items():
        print(f"Analizăm entitatea: {entity_name}")
        # Adăugăm o coloană pentru numărul de referințe
        col_name = f"{entity_name}_mentions"
        df[col_name] = df['full_text'].apply(
            lambda x: sum(1 for term in entity_terms if re.search(fr'\b{re.escape(term)}\b', str(x).lower()))
        )
        
        # Calculăm scorul de sentiment contextual pentru entitate
        df[f"{entity_name}_sentiment"] = df.apply(
            lambda row: calculate_entity_sentiment(row['full_text'], entity_terms, positive_terms, negative_terms), 
            axis=1
        )
        
        # Agregăm rezultatele pe orientare politică
        for political_leaning in df['political_leaning'].unique():
            if political_leaning == 'other' or pd.isna(political_leaning):
                continue
                
            group = df[df['political_leaning'] == political_leaning]
            
            # Calculăm procentul de postări care menționează entitatea
            total_posts = len(group)
            if total_posts == 0:
                continue
                
            mentions_posts = group[group[col_name] > 0].shape[0]
            mention_rate = (mentions_posts / total_posts) * 100
            
            # Calculăm sentimentul mediu față de entitate
            avg_sentiment = group[f"{entity_name}_sentiment"].mean()
            
            # Adăugăm rezultate doar dacă avem mențiuni
            if mentions_posts > 0:
                results.append({
                    'entity': entity_name,
                    'political_leaning': political_leaning,
                    'mention_rate': mention_rate,
                    'avg_sentiment': avg_sentiment,
                    'total_mentions': group[col_name].sum(),
                    'total_posts': total_posts
                })
    
    # Convertim rezultatele la DataFrame
    results_df = pd.DataFrame(results)
    
    if len(results_df) == 0:
        print("Nu sunt suficiente date pentru analiza geopolitică")
        return None
    
    # Salvăm rezultatele
    results_df.to_csv(os.path.join(geo_dir, 'geopolitical_attitudes.csv'), index=False)
    print(f"Atitudini geopolitice salvate în {os.path.join(geo_dir, 'geopolitical_attitudes.csv')}")
    
    # Creăm vizualizări
    # 1. Rata mențiunilor
    try:
        plt.figure(figsize=(14, 10))
        pivot_mentions = results_df.pivot(index='entity', columns='political_leaning', values='mention_rate')
        ax = pivot_mentions.plot(kind='barh', figsize=(14, 10))
        plt.title('Rata de menționare a entităților geopolitice (% din postări)', fontsize=16)
        plt.xlabel('Procent de postări care menționează entitatea', fontsize=14)
        plt.grid(axis='x', alpha=0.3)
        plt.legend(title='Orientare politică')
        plt.tight_layout()
        plt.savefig(os.path.join(geo_dir, 'geopolitical_mention_rates.png'))
        plt.close()
        print(f"Grafic de rata mențiunilor generat: {os.path.join(geo_dir, 'geopolitical_mention_rates.png')}")
    except Exception as e:
        print(f"Eroare la generarea graficului de mențiuni: {e}")
    
    # 2. Sentimentul mediu - varianta simplificată
    try:
        plt.figure(figsize=(14, 10))
        pivot_sentiment = results_df.pivot(index='entity', columns='political_leaning', values='avg_sentiment')

        # Folosim culorile standard pentru diferitele orientări politice
        ax = pivot_sentiment.plot(kind='barh', figsize=(14, 10))

        plt.title('Sentimentul mediu față de entități geopolitice', fontsize=16)
        plt.xlabel('Sentiment (negativ → pozitiv)', fontsize=14)
        plt.axvline(x=0, color='gray', linestyle='-', linewidth=0.8)
        plt.grid(axis='x', alpha=0.3)
        plt.legend(title='Orientare politică')
        plt.tight_layout()
        plt.savefig(os.path.join(geo_dir, 'geopolitical_sentiment.png'))
        plt.close()
        print(f"Grafic de sentiment generat: {os.path.join(geo_dir, 'geopolitical_sentiment.png')}")
    except Exception as e:
        print(f"Eroare la generarea graficului de sentiment: {e}")
    
    # Analizăm conflictele specifice în detaliu
    analyze_specific_conflicts(df, geo_dir)
    
    # Creăm un raport HTML
    create_geopolitical_report(results_df, geo_dir)
    
    return results_df



def analyze_specific_conflicts(df, output_dir):
    """Analizează în detaliu conflictele majore specifice"""
    
    # Definim conflictele specifice și termenii asociați - cu termeni extinși
    specific_conflicts = {
        'ukraine_russia': {
            'name': 'Conflictul Rusia-Ucraina',
            'terms': [
                'ukraine', 'ukrainian', 'zelensky', 'kyiv', 'kiev', 'kharkiv', 'odesa', 'crimea', 'donbas', 'luhansk', 'donetsk',
                'russia', 'russian', 'putin', 'moscow', 'kremlin', 'invasion', 'war in ukraine', 'special operation', 'eastern ukraine'
            ],
            'pro_ukraine': [
                'support ukraine', 'defend ukraine', 'ukrainian freedom', 'ukraine sovereignty', 'ukraine democracy',
                'stop russia', 'russian aggression', 'russian invasion', 'putin war', 'stand with ukraine', 'slava ukraini'
            ],
            'pro_russia': [
                'ukraine provocation', 'ukraine nazi', 'nato expansion', 'russia security', 'denazification',
                'donbas republic', 'russian minority', 'ukraine corrupt', 'biolab', 'ukraine coup', 'azov battalion'
            ]
        },
        'israel_palestine': {
            'name': 'Conflictul Israel-Palestina',
            'terms': [
                # Termeni mai generali pentru detectare mai permisivă
                'israel', 'israeli', 'netanyahu', 'idf', 'jerusalem', 'tel aviv', 'zionist', 'jewish state',
                'palestine', 'palestinian', 'gaza', 'hamas', 'west bank', 'fatah', 'middle east', 'holy land',
                # Termeni de conflict
                'settlements', 'occupied territories', 'two state', 'terrorism', 'intifada', 'ceasefire', 'peace process',
                # Locații relevante
                'gaza strip', 'dome of the rock', 'temple mount', 'golan heights', 'hebron', 'ramallah', 'bethlehem',
                # Termeni de conflict recenți
                'al aqsa', 'iron dome', 'right of return', 'october 7', '7th october', 'hamas attack', 
                'hostage', 'rocket', 'idf operation'
            ],
            'pro_israel': [
                # Termeni pro-Israel extins
                'support israel', 'defend israel', 'israel right', 'israel security', 'israel defense',
                'hamas terrorist', 'israel democracy', 'antisemitism', 'jewish state', 'right to defend',
                'terrorist attack', 'hostage rescue', 'october 7 attack', 'october 7 massacre', 'hamas hostage',
                'eliminate hamas', 'israel safety', 'israeli civilian', 'jewish homeland'
            ],
            'pro_palestine': [
                # Termeni pro-Palestina extins
                'free palestine', 'palestinian rights', 'israeli occupation', 'israeli apartheid', 'israeli colonialism',
                'illegal settlement', 'gaza blockade', 'palestinian civilian', 'right to return', 'ethnic cleansing',
                'gaza humanitarian', 'palestinian homeland', 'end occupation', 'gaza bombing', 'palestinian children',
                'israeli aggression', 'gaza civilian', 'refugee camp', 'idf brutality', 'ceasefire now'
            ]
        }
    }
    
    # Analizăm fiecare conflict
    for conflict_id, conflict_data in specific_conflicts.items():
        print(f"Analizăm conflictul: {conflict_data['name']}")
        
        # MODIFICARE 1: Detectare mai permisivă pentru Israel-Palestina
        if conflict_id == 'israel_palestine':
            # Folosește termeni individuali pentru potrivire mai permisivă
            df[f'{conflict_id}_mentioned'] = df['full_text'].apply(
                lambda x: any(term.lower() in str(x).lower() for term in conflict_data['terms'])
            )
        else:
            # Metoda originală pentru alte conflicte (pattern-matching exact)
            df[f'{conflict_id}_mentioned'] = df['full_text'].apply(
                lambda x: any(re.search(fr'\b{re.escape(term)}\b', str(x).lower()) for term in conflict_data['terms'])
            )
        
        # MODIFICARE 2: Detectare mai permisivă pentru termenii pro/contra Israel-Palestina
        if conflict_id == 'israel_palestine':
            # Termeni pentru fiecare parte
            side1_terms = conflict_data['pro_israel']
            side2_terms = conflict_data['pro_palestine']
            
            # Căutare simplificată
            df[f'{conflict_id}_pro_side1'] = df['full_text'].apply(
                lambda x: sum(1 for term in side1_terms if term.lower() in str(x).lower())
            )
            
            df[f'{conflict_id}_pro_side2'] = df['full_text'].apply(
                lambda x: sum(1 for term in side2_terms if term.lower() in str(x).lower())
            )
        else:
            # Metoda originală pentru alte conflicte
            side1_terms = conflict_data['pro_ukraine' if 'ukraine' in conflict_id else 'pro_israel']
            side2_terms = conflict_data['pro_russia' if 'ukraine' in conflict_id else 'pro_palestine']
            
            df[f'{conflict_id}_pro_side1'] = df['full_text'].apply(
                lambda x: sum(1 for term in side1_terms if re.search(fr'\b{re.escape(term)}\b', str(x).lower()))
            )
            
            df[f'{conflict_id}_pro_side2'] = df['full_text'].apply(
                lambda x: sum(1 for term in side2_terms if re.search(fr'\b{re.escape(term)}\b', str(x).lower()))
            )
        
        # Restul codului rămâne neschimbat...
        
        # Calculăm un scor de poziție (-1 = pro side2, 1 = pro side1)
        df[f'{conflict_id}_position'] = df.apply(
            lambda row: calculate_position_score(
                row[f'{conflict_id}_pro_side1'],
                row[f'{conflict_id}_pro_side2']
            ),
            axis=1
        )
        
        # MODIFICARE 3: Afișăm numărul de mențiuni detectate pentru a diagnostica problema
        mentions_count = df[df[f'{conflict_id}_mentioned'] == True].shape[0]
        print(f"Număr total de mențiuni pentru {conflict_data['name']}: {mentions_count}")
        
        # Definim etichetele pentru părți în funcție de conflict
        if conflict_id == 'ukraine_russia':
            side1 = "Ucraina"
            side2 = "Rusia"
        elif conflict_id == 'israel_palestine':
            side1 = "Israel"
            side2 = "Palestina"
        else:
            side1 = "Side1"
            side2 = "Side2"

        side1_count = df[df[f'{conflict_id}_pro_side1'] > 0].shape[0]
        side2_count = df[df[f'{conflict_id}_pro_side2'] > 0].shape[0]
        print(f"Poziții pro-{side1}: {side1_count}")
        print(f"Poziții pro-{side2}: {side2_count}")
        
        # Agregăm rezultatele pe orientare politică
        # Ignorăm valorile NaN și 'other'
        position_by_leaning = df[df['political_leaning'].notna() & (df['political_leaning'] != 'other')].groupby(
            'political_leaning')[f'{conflict_id}_position'].mean().reset_index()
            
        mention_rate_by_leaning = df[df['political_leaning'].notna() & (df['political_leaning'] != 'other')].groupby(
            'political_leaning')[f'{conflict_id}_mentioned'].mean().reset_index()
            
        mention_rate_by_leaning[f'{conflict_id}_mentioned'] *= 100  # Convert to percentage
        
        # MODIFICARE 4: O verificare mai detaliată dacă avem suficiente date
        if len(position_by_leaning) == 0 or len(mention_rate_by_leaning) == 0:
            print(f"Nu sunt suficiente date pentru analiza conflictului {conflict_data['name']}")
            continue
            
        if mention_rate_by_leaning[f'{conflict_id}_mentioned'].max() < 1.0:
            print(f"Rata de menționare prea mică pentru conflictul {conflict_data['name']} (<1%)")
            
            # MODIFICARE 5: Încercăm să generăm totuși graficele, chiar dacă avem puține date
            if mention_rate_by_leaning[f'{conflict_id}_mentioned'].sum() == 0:
                print(f"Nu s-au găsit mențiuni pentru {conflict_data['name']}. Sărim la următorul conflict.")
                continue
        # Verificăm dacă avem date
        if len(position_by_leaning) == 0 or len(mention_rate_by_leaning) == 0:
            print(f"Nu sunt suficiente date pentru analiza conflictului {conflict_data['name']}")
            continue
        
        # Salvăm rezultatele
        position_by_leaning.to_csv(os.path.join(output_dir, f'{conflict_id}_positions.csv'), index=False)
        print(f"Poziții salvate în {os.path.join(output_dir, f'{conflict_id}_positions.csv')}")
        
        # Vizualizăm rezultatele
        try:
            # Definim o paletă de culori care conține toate orientările politice posibile
            palette_dict = {'conservative': 'crimson', 'liberal': 'royalblue', 'moderate': 'purple', 
                           'libertarian': 'gold', 'green': 'green', 'independent': 'gray'}
            
            # Filtrăm doar orientările prezente în date
            available_leanings = position_by_leaning['political_leaning'].unique()
            current_palette = {k: v for k, v in palette_dict.items() if k in available_leanings}
            
            plt.figure(figsize=(10, 6))
            sns.barplot(
                data=position_by_leaning, 
                x='political_leaning', 
                y=f'{conflict_id}_position',
                palette=current_palette
            )
            
            # Adăugăm titluri și etichete
            side1 = "pro-Ucraina" if "ukraine" in conflict_id else "pro-Israel"
            side2 = "pro-Rusia" if "ukraine" in conflict_id else "pro-Palestina"
            
            plt.axhline(y=0, color='gray', linestyle='-', linewidth=0.8)
            plt.title(f'Poziție medie în {conflict_data["name"]} pe orientare politică', fontsize=16)
            plt.ylabel(f'Poziție ({side2} ← 0 → {side1})', fontsize=14)
            plt.xlabel('Orientare politică', fontsize=14)
            plt.ylim(-1, 1)
            plt.tight_layout()
            plt.savefig(os.path.join(output_dir, f'{conflict_id}_positions.png'))
            plt.close()
            print(f"Grafic de poziții generat: {os.path.join(output_dir, f'{conflict_id}_positions.png')}")
            
            # Vizualizăm rata de menționare
            plt.figure(figsize=(10, 6))
            sns.barplot(
                data=mention_rate_by_leaning, 
                x='political_leaning', 
                y=f'{conflict_id}_mentioned',
                palette=current_palette
            )
            plt.title(f'Rata de menționare a conflictului {conflict_data["name"]}', fontsize=16)
            plt.ylabel('% postări care menționează conflictul', fontsize=14)
            plt.xlabel('Orientare politică', fontsize=14)
            plt.tight_layout()
            plt.savefig(os.path.join(output_dir, f'{conflict_id}_mention_rate.png'))
            plt.close()
            print(f"Grafic de rata menționărilor generat: {os.path.join(output_dir, f'{conflict_id}_mention_rate.png')}")
            
        except Exception as e:
            print(f"Eroare la generarea graficelor pentru {conflict_id}: {e}")
        
        # Extragem exemple de postări
        extract_example_posts(df, conflict_id, output_dir)

# Asigură-te că ai descărcat stopwords
nltk.download('stopwords', download_dir=os.path.join(os.getcwd(), "nltk_data"))

def extract_example_posts(df, conflict_id, output_dir, n=5):
    """
    Extrage exemple de postări relevante pentru un conflict și le salvează într-un fișier CSV.
    """
    # Selectăm postările care menționează conflictul
    mention_col = f"{conflict_id}_mentioned"
    if mention_col not in df.columns:
        print(f"Coloana {mention_col} nu există în DataFrame.")
        return

    examples = df[df[mention_col] == True].copy()
    if examples.empty:
        print(f"Nu există postări relevante pentru {conflict_id}.")
        return

    # Selectăm doar câteva exemple pentru fiecare orientare politică
    example_rows = []
    for leaning in examples['political_leaning'].dropna().unique():
        group = examples[examples['political_leaning'] == leaning].head(n)
        example_rows.append(group)

    result = pd.concat(example_rows)
    # Salvăm exemplele într-un fișier CSV
    result[['political_leaning', 'title', 'content', 'full_text']].to_csv(
        os.path.join(output_dir, f"{conflict_id}_example_posts.csv"), index=False
    )
    print(f"Exemple de postări pentru {conflict_id} salvate în {os.path.join(output_dir, f'{conflict_id}_example_posts.csv')}")

def analyze_political_language_differences(output_dir):
    """
    Analizează diferențele de limbaj între diferite grupuri politice.
    Identifică termeni distinctivi, vulgaritate, și pattern-uri lexicale.
    """
    
    # Creăm directorul pentru rezultate
    language_dir = os.path.join(output_dir, "language_analysis")
    os.makedirs(language_dir, exist_ok=True)
    
    # Încărcăm datele
    posts_file = os.path.join(output_dir, "all_posts.csv")
    if not os.path.exists(posts_file):
        print(f"Fișierul {posts_file} nu există.")
        return None
    
    df_posts = pd.read_csv(posts_file)
    
    # Definim subreddit-uri și grupuri politice
    conservative_subreddits = ['Conservative', 'Republican', 'conservatives', 'TheDonald']
    liberal_subreddits = ['democrats', 'Liberal', 'progressive']
    
    # Adăugăm coloana cu orientarea politică
    df_posts['political_leaning'] = 'other'
    df_posts.loc[df_posts['subreddit'].isin(conservative_subreddits), 'political_leaning'] = 'conservative'
    df_posts.loc[df_posts['subreddit'].isin(liberal_subreddits), 'political_leaning'] = 'liberal'
    
    # Filtrăm doar postările cu orientare politică clară
    df_political = df_posts[df_posts['political_leaning'] != 'other'].copy()
    
    # Combinăm titlul și conținutul pentru analiză
    df_political['full_text'] = df_political['title'] + " " + df_political['content'].fillna("")
    
    # Analizăm vulgaritatea și agresivitatea
    print("Analizăm limbajul vulgar și agresiv...")
    analyze_vulgar_language(df_political, language_dir)
    
    # Identificăm termeni distinctivi pentru fiecare tabără politică
    print("Identificăm termeni distinctivi...")
    identify_distinctive_terms(df_political, language_dir)
    
    # Analizăm entități specifice (războaie, conflicte, politici externe)
    print("Analizăm referințele la conflicte și politică externă...")
    analyze_conflict_references(df_political, language_dir)
    
    print("Analizăm atitudinile geopolitice (Ucraina, Israel, etc.)...")
    analyze_geopolitical_attitudes(df_political, language_dir)
        
        # Creăm raportul
    create_language_report(df_political, language_dir)
    

def calculate_entity_sentiment(text, entity_terms, positive_terms, negative_terms):
    """Calculează sentimentul în contextul unei entități geopolitice specifice"""
    
    # Verifică dacă textul este valid
    if pd.isna(text) or not isinstance(text, str) or not text:
        return 0
    
    text = text.lower()
    
    # Verificăm dacă entitatea este menționată
    if not any(re.search(fr'\b{re.escape(term)}\b', text) for term in entity_terms):
        return 0
    
    # Divizăm textul în propoziții
    sentences = re.split(r'[.!?]', text)
    
    # Filtrăm doar propozițiile care menționează entitatea
    relevant_sentences = [s for s in sentences if any(
        re.search(fr'\b{re.escape(term)}\b', s) for term in entity_terms
    )]
    
    if not relevant_sentences:
        return 0
    
    # Calculăm scorul de sentiment pentru fiecare propoziție relevantă
    total_score = 0
    
    for sentence in relevant_sentences:
        sentence_score = 0
        
        # Verificăm termenii pozitivi
        for term, score in positive_terms.items():
            if re.search(fr'\b{re.escape(term)}\b', sentence):
                sentence_score += score
        
        # Verificăm termenii negativi
        for term, score in negative_terms.items():
            if re.search(fr'\b{re.escape(term)}\b', sentence):
                sentence_score += score
        
        total_score += sentence_score
    
    # Normalizăm scorul la intervalul [-1, 1]
    max_possible_score = max(3 * len(relevant_sentences), 1)  # Presupunem că scorul maxim per propoziție este 3
    normalized_score = total_score / max_possible_score
    
    return max(min(normalized_score, 1), -1)  # Limităm la intervalul [-1, 1]

def calculate_position_score(pro_side1, pro_side2):
    """Calculează un scor de poziție între -1 (pro side2) și 1 (pro side1)"""
    
    if pro_side1 == 0 and pro_side2 == 0:
        return 0
    
    return (pro_side1 - pro_side2) / max(pro_side1 + pro_side2, 1)

def analyze_vulgar_language(df, output_dir):
    """Analizează prezența limbajului vulgar și agresiv în diferite grupuri politice."""
    
    # Definim liste de cuvinte vulgare și agresive
    # Notă: În practica reală, ar trebui folosite resurse externe mai complete
    vulgar_words = [
        'fuck', 'shit', 'ass', 'damn', 'bitch', 'crap', 'hell',
        'bastard', 'asshole', 'bullshit', 'cunt', 'dick', 'piss',
        'whore', 'slut', 'douchebag', 'idiot', 'moron', 'stupid'
    ]
    
    aggressive_words = [
        'kill', 'destroy', 'attack', 'fight', 'hate', 'war', 'battle',
        'enemy', 'defeat', 'crush', 'violent', 'violence', 'threat',
        'threaten', 'aggressive', 'hostile', 'invade', 'invasion',
        'combat', 'weapon', 'missile', 'bomb', 'nuke', 'punch'
    ]
    
    # Calculăm scorul de vulgaritate pentru fiecare postare
    df['vulgar_score'] = df['full_text'].apply(
        lambda x: sum(1 for word in str(x).lower().split() if word in vulgar_words)
    )
    
    df['aggressive_score'] = df['full_text'].apply(
        lambda x: sum(1 for word in str(x).lower().split() if word in aggressive_words)
    )
    
    # Normalizăm scorurile în funcție de lungimea textului
    df['text_length'] = df['full_text'].apply(lambda x: len(str(x).split()))
    df['vulgar_rate'] = df['vulgar_score'] / df['text_length'] * 100
    df['aggressive_rate'] = df['aggressive_score'] / df['text_length'] * 100
    
    # Analizăm scorurile pe grupuri politice
    vulgar_by_leaning = df.groupby('political_leaning')[['vulgar_rate', 'aggressive_rate']].mean()
    vulgar_by_subreddit = df.groupby('subreddit')[['vulgar_rate', 'aggressive_rate']].mean().sort_values('vulgar_rate', ascending=False)
    
    # Salvăm rezultatele
    vulgar_by_leaning.to_csv(os.path.join(output_dir, 'vulgar_by_political_leaning.csv'))
    vulgar_by_subreddit.to_csv(os.path.join(output_dir, 'vulgar_by_subreddit.csv'))
    
    # Vizualizăm rezultatele
    # 1. Comparație între orientări politice
    plt.figure(figsize=(10, 6))
    vulgar_by_leaning.plot(kind='bar', figsize=(10, 6))
    plt.title('Rata limbajului vulgar și agresiv pe orientare politică', fontsize=16)
    plt.ylabel('Procent de cuvinte (%))', fontsize=14)
    plt.xticks(rotation=0)
    plt.grid(axis='y', alpha=0.3)
    plt.savefig(os.path.join(output_dir, 'vulgar_language_by_leaning.png'))
    plt.close()
    
    # 2. Top subreddit-uri după limbaj vulgar
    plt.figure(figsize=(12, 8))
    vulgar_by_subreddit.head(10)['vulgar_rate'].plot(kind='bar', color='crimson')
    plt.title('Top 10 Subreddit-uri după rata limbajului vulgar', fontsize=16)
    plt.ylabel('Rata limbajului vulgar (%)', fontsize=14)
    plt.xticks(rotation=45, ha='right')
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'vulgar_rate_by_subreddit.png'))
    plt.close()
    
    # 3. Top subreddit-uri după limbaj agresiv
    plt.figure(figsize=(12, 8))
    vulgar_by_subreddit.head(10)['aggressive_rate'].plot(kind='bar', color='darkred')
    plt.title('Top 10 Subreddit-uri după rata limbajului agresiv', fontsize=16)
    plt.ylabel('Rata limbajului agresiv (%)', fontsize=14)
    plt.xticks(rotation=45, ha='right')
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'aggressive_rate_by_subreddit.png'))
    plt.close()
    
    return vulgar_by_leaning, vulgar_by_subreddit

def identify_distinctive_terms(df, output_dir):
    """Identifică termenii care sunt cei mai distinctivi pentru fiecare orientare politică."""
    
    # Verificăm dacă avem date suficiente
    if len(df) < 10:
        print("Prea puține date pentru analiza termenilor distinctivi")
        return None, None
    
    # Extragem texte pentru fiecare orientare
    conservative_texts = df[df['political_leaning'] == 'conservative']['full_text'].fillna("").tolist()
    liberal_texts = df[df['political_leaning'] == 'liberal']['full_text'].fillna("").tolist()
    
    if not conservative_texts or not liberal_texts:
        print("Nu sunt suficiente date pentru ambele orientări politice")
        return None, None
    
    # Combinăm toate textele într-un singur document pentru fiecare orientare
    combined_conservative = " ".join(conservative_texts)
    combined_liberal = " ".join(liberal_texts)
    
    # Utilizăm CountVectorizer cu parametri care vor funcționa întotdeauna
    # pentru doar 2 documente (combined_conservative și combined_liberal)
    vectorizer = CountVectorizer(
        min_df=1,          # Apare în cel puțin 1 document
        max_df=1.0,        # Poate apărea în până la 100% din documente
        ngram_range=(1, 2),  # Include unigrame și bigrame
        stop_words='english'
    )
    
    # Vectorizăm cele două texte combinate
    X = vectorizer.fit_transform([combined_conservative, combined_liberal])
    feature_names = vectorizer.get_feature_names_out()
    
    # Extragem frecvențele
    conservative_counts = X[0].toarray()[0]
    liberal_counts = X[1].toarray()[0]
    
    # Calculăm raportul (cu epsilon pentru a evita împărțirea la zero)
    epsilon = 1e-10
    conservative_ratio = (conservative_counts + epsilon) / (liberal_counts + epsilon)
    liberal_ratio = (liberal_counts + epsilon) / (conservative_counts + epsilon)
    
    # Extragem termenii distinctivi
    # Limităm numărul la maximum numărul de caracteristici disponibile
    top_n = min(100, len(feature_names))
    
    # Sortăm și extragem indecși
    conservative_indices = np.argsort(conservative_ratio)[-top_n:][::-1]  # Inversăm pentru a avea descrescător
    liberal_indices = np.argsort(liberal_ratio)[-top_n:][::-1]
    
    # Creăm listele cu termeni distinctivi
    conservative_terms = [(feature_names[i], conservative_ratio[i], conservative_counts[i], liberal_counts[i]) 
                         for i in conservative_indices]
    
    liberal_terms = [(feature_names[i], liberal_ratio[i], liberal_counts[i], conservative_counts[i]) 
                    for i in liberal_indices]
    
    # Convertim la DataFrames
    conservative_df = pd.DataFrame(conservative_terms, 
                                  columns=['term', 'ratio', 'conservative_count', 'liberal_count'])
    
    liberal_df = pd.DataFrame(liberal_terms, 
                             columns=['term', 'ratio', 'liberal_count', 'conservative_count'])
    
    # Salvăm termenii distinctivi
    conservative_df.to_csv(os.path.join(output_dir, 'conservative_distinctive_terms.csv'), index=False)
    liberal_df.to_csv(os.path.join(output_dir, 'liberal_distinctive_terms.csv'), index=False)
    
    # Creăm vizualizările pentru termenii distinctivi
    try:
        # Conservative word cloud
        conservative_word_dict = {row['term']: row['ratio'] for _, row in conservative_df.head(50).iterrows()}
        plt.figure(figsize=(12, 8))
        wc = WordCloud(
            background_color='white',
            width=800, height=600,
            max_words=100,
            prefer_horizontal=0.9,
            colormap='Reds'
        ).generate_from_frequencies(conservative_word_dict)
        
        plt.imshow(wc, interpolation='bilinear')
        plt.axis("off")
        plt.title('Termeni distinctivi pentru orientarea conservatoare', fontsize=16)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'conservative_distinctive_terms.png'))
        plt.close()
        
        # Liberal word cloud
        liberal_word_dict = {row['term']: row['ratio'] for _, row in liberal_df.head(50).iterrows()}
        plt.figure(figsize=(12, 8))
        wc = WordCloud(
            background_color='white',
            width=800, height=600,
            max_words=100,
            prefer_horizontal=0.9,
            colormap='Blues'
        ).generate_from_frequencies(liberal_word_dict)
        
        plt.imshow(wc, interpolation='bilinear')
        plt.axis("off")
        plt.title('Termeni distinctivi pentru orientarea liberală', fontsize=16)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'liberal_distinctive_terms.png'))
        plt.close()
        
        # Bar charts
        plt.figure(figsize=(14, 10))
        sns.barplot(data=conservative_df.head(20), x='ratio', y='term')
        plt.title('Top 20 termeni distinctivi pentru orientarea conservatoare', fontsize=16)
        plt.xlabel('Raportul de utilizare (conservator / liberal)', fontsize=14)
        plt.grid(axis='x', alpha=0.3)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'conservative_top_terms.png'))
        plt.close()
        
        plt.figure(figsize=(14, 10))
        sns.barplot(data=liberal_df.head(20), x='ratio', y='term')
        plt.title('Top 20 termeni distinctivi pentru orientarea liberală', fontsize=16)
        plt.xlabel('Raportul de utilizare (liberal / conservator)', fontsize=14)
        plt.grid(axis='x', alpha=0.3)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'liberal_top_terms.png'))
        plt.close()
    except Exception as e:
        print(f"Eroare la crearea vizualizărilor: {e}")
    
    return conservative_df, liberal_df

def analyze_conflict_references(df, output_dir):
    """Analizează referințele la conflicte, războaie și probleme geopolitice."""
    
    # Definim categorii de termeni legați de conflicte și politică externă
    conflict_categories = {
        'war': ['war', 'battle', 'combat', 'troops', 'invasion', 'invade', 'military', 'army', 
                'soldiers', 'weapons', 'tanks', 'missiles', 'airstrikes', 'bombing'],
        'nuclear': ['nuclear', 'nuke', 'atomic', 'radiation', 'fallout', 'warhead', 'missile'],
        'terrorism': ['terror', 'terrorist', 'terrorism', 'attack', 'bomb', 'bombing', 'isis', 'al-qaeda'],
        'sanctions': ['sanctions', 'embargo', 'economic pressure', 'ban', 'restrict', 'blockade'],
        'diplomacy': ['diplomacy', 'diplomatic', 'negotiations', 'peace', 'treaty', 'agreement', 'talks',
                    'alliance', 'allies', 'nato', 'un', 'united nations']
    }
    
    # Calculăm frecvența pentru fiecare categorie
    for category, terms in conflict_categories.items():
        df[f'{category}_mentions'] = df['full_text'].apply(
            lambda x: sum(1 for term in terms if re.search(r'\b' + term + r'\b', str(x).lower()))
        )
        
        # Normalizăm cu lungimea textului
        df[f'{category}_rate'] = df[f'{category}_mentions'] / df['text_length'] * 100
    
    # Analizăm diferențele între orientări politice
    conflict_cols = [f'{cat}_rate' for cat in conflict_categories.keys()]
    conflict_by_leaning = df.groupby('political_leaning')[conflict_cols].mean()
    
    # Salvăm rezultatele
    conflict_by_leaning.to_csv(os.path.join(output_dir, 'conflict_references_by_leaning.csv'))
    
    # Vizualizăm rezultatele
    plt.figure(figsize=(12, 8))
    conflict_by_leaning.T.plot(kind='bar', figsize=(12, 8))
    plt.title('Rata mențiunilor legate de conflicte și politică externă pe orientare politică', fontsize=16)
    plt.ylabel('Rata mențiunilor (%)', fontsize=14)
    plt.xticks(rotation=45, ha='right')
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'conflict_references_by_leaning.png'))
    plt.close()
    
    # Analizăm contextul în care apar referințele la război
    analyze_war_context(df, output_dir)
    
    return conflict_by_leaning

def analyze_war_context(df, output_dir):
    """Analizează contextul în care apar termenii legați de război."""
    
    # Extragem propoziții care conțin termeni legați de război
    war_terms = ['war', 'battle', 'combat', 'troops', 'invasion', 'invade', 'military']
    war_sentences = []
    
    for _, row in df.iterrows():
        text = str(row['full_text']).lower()
        sentences = re.split(r'[.!?]', text)
        
        for sentence in sentences:
            if any(re.search(r'\b' + term + r'\b', sentence) for term in war_terms):
                war_sentences.append({
                    'sentence': sentence.strip(),
                    'political_leaning': row['political_leaning'],
                    'subreddit': row['subreddit']
                })
    
    if war_sentences:
        # Convertim la DataFrame
        war_context_df = pd.DataFrame(war_sentences)
        
        # Salvăm exemplele de contexte
        war_context_df.to_csv(os.path.join(output_dir, 'war_contexts.csv'), index=False)
        
        # Analizăm cuvintele care apar frecvent în contextul războiului
        # Ajustăm min_df și max_df pentru a evita ValueError când avem doar 2 documente
        vectorizer = CountVectorizer(
            stop_words='english',
            min_df=1,
            max_df=1.0
        )
        
        # Separat pentru fiecare orientare politică
        conservative_contexts = ' '.join(war_context_df[war_context_df['political_leaning'] == 'conservative']['sentence'])
        liberal_contexts = ' '.join(war_context_df[war_context_df['political_leaning'] == 'liberal']['sentence'])
        
        X = vectorizer.fit_transform([conservative_contexts, liberal_contexts])
        feature_names = vectorizer.get_feature_names_out()
        
        conservative_counts = X[0].toarray()[0]
        liberal_counts = X[1].toarray()[0]
        
        # Top cuvinte în contextul războiului pentru fiecare orientare
        conservative_top = [(feature_names[i], conservative_counts[i]) 
                           for i in conservative_counts.argsort()[-30:][::-1]]
        liberal_top = [(feature_names[i], liberal_counts[i]) 
                      for i in liberal_counts.argsort()[-30:][::-1]]
        
        # Vizualizăm rezultatele
        plt.figure(figsize=(12, 8))
        plt.bar(
            [item[0] for item in conservative_top[:15]],
            [item[1] for item in conservative_top[:15]],
            color='crimson'
        )
        plt.title('Top cuvinte în contextul războiului - Conservatori', fontsize=16)
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'conservative_war_context_words.png'))
        plt.close()
        
        plt.figure(figsize=(12, 8))
        plt.bar(
            [item[0] for item in liberal_top[:15]],
            [item[1] for item in liberal_top[:15]],
            color='royalblue'
        )
        plt.title('Top cuvinte în contextul războiului - Liberali', fontsize=16)
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'liberal_war_context_words.png'))
        plt.close()

def create_language_report(df, output_dir):
    """Creează un raport HTML cu constatările analizei limbajului."""
    
    html = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Analiza Diferențelor de Limbaj în Discursul Politic</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                line-height: 1.6;
                margin: 0;
                padding: 20px;
                color: #333;
                max-width: 1200px;
                margin: 0 auto;
            }
            h1, h2, h3 {
                color: #2c3e50;
                margin-top: 1.5em;
            }
            .container {
                margin-bottom: 40px;
            }
            .visualization {
                margin: 20px 0;
                text-align: center;
            }
            .visualization img {
                max-width: 100%;
                box-shadow: 0 4px 8px rgba(0,0,0,0.1);
            }
            .highlight {
                background-color: #f8f9fa;
                padding: 15px;
                border-left: 4px solid #e74c3c;
                margin: 20px 0;
            }
            .row {
                display: flex;
                flex-wrap: wrap;
            }
            .col {
                flex: 1;
                min-width: 300px;
                margin: 10px;
            }
            .conservative {
                color: #c0392b;
                font-weight: bold;
            }
            .liberal {
                color: #2980b9;
                font-weight: bold;
            }
        </style>
    </head>
    <body>
        <h1>Analiza Diferențelor de Limbaj în Discursul Politic</h1>
        
        <div class="highlight">
            <p>Această analiză explorează modul în care diferite orientări politice utilizează limbajul, cu accent pe termenii distinctivi, limbajul vulgar sau agresiv, și referințele la conflicte sau politică externă.</p>
        </div>
        
        <div class="container">
            <h2>Termeni Distinctivi pe Orientare Politică</h2>
            <p>Word cloud-urile de mai jos ilustrează termenii cel mai puternic asociați cu fiecare orientare politică:</p>
            
            <div class="row">
                <div class="col">
                    <h3>Termeni <span class="conservative">Conservatori</span></h3>
                    <div class="visualization">
                        <img src="conservative_distinctive_terms.png" alt="Termeni distinctivi conservatori">
                    </div>
                </div>
                <div class="col">
                    <h3>Termeni <span class="liberal">Liberali</span></h3>
                    <div class="visualization">
                        <img src="liberal_distinctive_terms.png" alt="Termeni distinctivi liberali">
                    </div>
                </div>
            </div>
            
            <div class="row">
                <div class="col">
                    <div class="visualization">
                        <img src="conservative_top_terms.png" alt="Top termeni conservatori">
                        <p>Top termeni distinctivi pentru orientarea conservatoare</p>
                    </div>
                </div>
                <div class="col">
                    <div class="visualization">
                        <img src="liberal_top_terms.png" alt="Top termeni liberali">
                        <p>Top termeni distinctivi pentru orientarea liberală</p>
                    </div>
                </div>
            </div>
        </div>
        
        <div class="container">
            <h2>Limbaj Vulgar și Agresiv</h2>
            <p>Comparația utilizării limbajului vulgar și agresiv între grupurile politice:</p>
            
            <div class="visualization">
                <img src="vulgar_language_by_leaning.png" alt="Limbaj vulgar pe orientare politică">
                <p>Rata medie a limbajului vulgar și agresiv pe orientare politică</p>
            </div>
            
            <div class="row">
                <div class="col">
                    <div class="visualization">
                        <img src="vulgar_rate_by_subreddit.png" alt="Vulgaritate pe subreddit">
                        <p>Top subreddit-uri după rata limbajului vulgar</p>
                    </div>
                </div>
                <div class="col">
                    <div class="visualization">
                        <img src="aggressive_rate_by_subreddit.png" alt="Agresivitate pe subreddit">
                        <p>Top subreddit-uri după rata limbajului agresiv</p>
                    </div>
                </div>
            </div>
        </div>
        
        <div class="container">
            <h2>Referințe la Conflicte și Politică Externă</h2>
            <p>Analiza modului în care diferitele grupuri politice discută despre conflicte, războaie și politică externă:</p>
            
            <div class="visualization">
                <img src="conflict_references_by_leaning.png" alt="Referințe la conflicte pe orientare politică">
                <p>Rata mențiunilor legate de diverse aspecte ale conflictelor și politicii externe</p>
            </div>
            
            <div class="row">
                <div class="col">
                    <div class="visualization">
                        <img src="conservative_war_context_words.png" alt="Context de război - conservatori">
                        <p>Top cuvinte folosite de conservatori în contextul discuțiilor despre război</p>
                    </div>
                </div>
                <div class="col">
                    <div class="visualization">
                        <img src="liberal_war_context_words.png" alt="Context de război - liberali">
                        <p>Top cuvinte folosite de liberali în contextul discuțiilor despre război</p>
                    </div>
                </div>
            </div>
        </div>
        
        <div class="container">
            <h2>Concluzii</h2>
            <p>Analiza diferențelor de limbaj între orientările politice relevă:</p>
            <ul>
                <li><strong>Vocabular distinct</strong>: Fiecare orientare politică folosește un set de termeni distinctivi care reflectă prioritățile și cadrele lor conceptuale.</li>
                <li><strong>Diferențe în registrul limbajului</strong>: Există variații semnificative în utilizarea limbajului vulgar și agresiv între diferite comunități politice.</li>
                <li><strong>Abordări diferite ale conflictelor</strong>: Modul în care se discută despre războaie, politică externă și conflicte variază considerabil între grupurile politice, cu accent pe aspecte și termeni diferiți.</li>
            </ul>
            <p>Aceste pattern-uri lingvistice ne ajută să înțelegem mai bine cum diferitele orientări politice conceptualizează și comunică despre probleme importante, și cum limbajul folosit poate reflecta valorile și prioritățile diferitelor grupuri.</p>
        </div>
    </body>
    </html>
    """
    
    # Salvăm raportul HTML
    with open(os.path.join(output_dir, 'language_analysis_report.html'), 'w', encoding='utf-8') as f:
        f.write(html)
        
    print(f"✅ Raport de analiză a limbajului generat în {output_dir}")

# Rulare
if __name__ == "__main__":
    output_dir = "us_politics_data_20250616_094313"
    analyze_political_language_differences(output_dir)
 

[nltk_data] Downloading package stopwords to c:\Users\OWNER\Desktop\OP
[nltk_data]     SWAT\Prezentare\ChatBot\Server\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Note: you may need to restart the kernel to use updated packages.
Analizăm limbajul vulgar și agresiv...
Identificăm termeni distinctivi...
Analizăm referințele la conflicte și politică externă...
Analizăm atitudinile geopolitice (Ucraina, Israel, etc.)...
Analizăm entitatea: ukraine
Analizăm entitatea: russia
Analizăm entitatea: israel
Analizăm entitatea: palestine
Analizăm entitatea: iran
Analizăm entitatea: china
Analizăm entitatea: nato
Atitudini geopolitice salvate în us_politics_data_20250616_094313\language_analysis\geopolitical_analysis\geopolitical_attitudes.csv
Grafic de rata mențiunilor generat: us_politics_data_20250616_094313\language_analysis\geopolitical_analysis\geopolitical_mention_rates.png
Grafic de sentiment generat: us_politics_data_20250616_094313\language_analysis\geopolitical_analysis\geopolitical_sentiment.png
Analizăm conflictul: Conflictul Rusia-Ucraina
Număr total de mențiuni pentru Conflictul Rusia-Ucraina: 263
Poziții pro-Ucraina: 24
Poziții pro-Rusia: 0
P

C:\Users\OWNER\AppData\Local\Temp\ipykernel_25792\2509978551.py:520: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(
C:\Users\OWNER\AppData\Local\Temp\ipykernel_25792\2509978551.py:543: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(


Grafic de rata menționărilor generat: us_politics_data_20250616_094313\language_analysis\geopolitical_analysis\ukraine_russia_mention_rate.png
Exemple de postări pentru ukraine_russia salvate în us_politics_data_20250616_094313\language_analysis\geopolitical_analysis\ukraine_russia_example_posts.csv
Analizăm conflictul: Conflictul Israel-Palestina
Număr total de mențiuni pentru Conflictul Israel-Palestina: 249
Poziții pro-Israel: 7
Poziții pro-Palestina: 1
Poziții salvate în us_politics_data_20250616_094313\language_analysis\geopolitical_analysis\israel_palestine_positions.csv
Grafic de poziții generat: us_politics_data_20250616_094313\language_analysis\geopolitical_analysis\israel_palestine_positions.png


C:\Users\OWNER\AppData\Local\Temp\ipykernel_25792\2509978551.py:520: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(
C:\Users\OWNER\AppData\Local\Temp\ipykernel_25792\2509978551.py:543: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(


Grafic de rata menționărilor generat: us_politics_data_20250616_094313\language_analysis\geopolitical_analysis\israel_palestine_mention_rate.png
Exemple de postări pentru israel_palestine salvate în us_politics_data_20250616_094313\language_analysis\geopolitical_analysis\israel_palestine_example_posts.csv
✅ Raport de analiză geopolitică generat în us_politics_data_20250616_094313\language_analysis\geopolitical_analysis
✅ Raport de analiză a limbajului generat în us_politics_data_20250616_094313\language_analysis


<Figure size 1000x600 with 0 Axes>

<Figure size 1200x800 with 0 Axes>

<Figure size 1400x1000 with 0 Axes>

<Figure size 1400x1000 with 0 Axes>

In [7]:
!pip install wordcloud


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: C:\Users\OWNER\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [12]:
!python -m spacy download en_core_web_sm


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 660.6 kB/s eta 0:00:20
     --------------------------------------- 0.1/12.8 MB 980.4 kB/s eta 0:00:13
      --------------------------------------- 0.3/12.8 MB 1.8 MB/s eta 0:00:07
     - -------------------------------------- 0.6/12.8 MB 2.9 MB/s eta 0:00:05
     -- ------------------------------------- 0.8/12.8 MB 3.5 MB/s eta 0:00:04
     --- ------------------------------------ 1.0/12.8 MB 3.7 MB/s eta 0:00:04
     --- ------------------------------------ 1.2/12.8 MB 3.7 MB/s eta 0:00:04
     ---- ----------------------------------- 1.4/12.8 MB 3.7 MB/s eta 0:00:04
     ---- ----------------------------------- 1.5/12.8 MB 3.5 MB/s eta 0:00:04
     ----- ---------------------------------- 1.7/12.8 MB 3.6 MB/s eta 0:00:04
     ------ --------------------------------- 2.0/12.8 MB 3.9 MB/s eta 0:00:03
     ------- -------------------------------- 2.3/12.8 MB


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
